In [199]:
import pandas as pd
import numpy as np
import random

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from functools import partial

# 1. Load data


In [2]:
data_path = "ChallengeXHEC23022024.xlsx"
dict_df_sheets = pd.read_excel(data_path, sheet_name=None)
df_hist_jan = dict_df_sheets["JAN24"].copy()
df_clients = dict_df_sheets["clients"].copy()
df_inter = dict_df_sheets["intervenants"].copy()

# 2. Create matrixes to have true travel time mappings


In [4]:
c_c_dist = pd.read_csv("client_client_distance.csv")
c_l_need = pd.read_csv("client_loc_need.csv")
c_n_dist = pd.read_csv("client_nurse_distance.csv")

list_clients_1 = c_c_dist["ID Client 1"].unique()
list_clients_2 = c_c_dist["ID Client 2"].unique()
full_client_list = list(set([*list_clients_1, *list_clients_2]))
nb_full_client = len(full_client_list)

c_l_need.drop(columns=["Unnamed: 0", "Latitude", "Longitude"], inplace=True)

In [5]:
client_id_to_idx = {}
client_idx_to_id = {}
for k in range(len(full_client_list)):
    client_id_to_idx[full_client_list[k]] = k
    client_idx_to_id[k] = full_client_list[k]

In [6]:
def convert_duration(str_duration):
    if "hour" in str_duration:
        list = str_duration.split()
        nb_minutes = int(list[0]) * 60 + int(list[2])
        return nb_minutes
    else:
        list = str_duration.split()
        nb_minutes = int(list[0])
        return nb_minutes

In [7]:
def define_client_time_matrix(
    df, vehicle_type, client_id_to_idx=client_id_to_idx, out_value=4000
):
    list_clients_1 = df["ID Client 1"].unique()
    list_clients_2 = df["ID Client 2"].unique()
    full_list = list(set([*list_clients_1, *list_clients_2]))

    matrix = [[0 for i in range(len(full_list))] for k in range(len(full_list))]
    col_type = "Duration_" + vehicle_type
    for client_1_id in full_list:
        client_1_idx = client_id_to_idx[client_1_id]
        for client_2_id in full_list:
            client_2_idx = client_id_to_idx[client_2_id]
            if client_1_id != client_2_id:
                try:
                    travel_time = convert_duration(
                        df.loc[
                            (df["ID Client 1"] == client_1_id)
                            & (df["ID Client 2"] == client_2_id),
                            col_type,
                        ].values[0]
                    )
                    matrix[client_1_idx][client_2_idx] = travel_time
                except IndexError:
                    try:
                        travel_time = convert_duration(
                            df.loc[
                                (df["ID Client 1"] == client_2_id)
                                & (df["ID Client 2"] == client_1_id),
                                col_type,
                            ].values[0]
                        )
                        matrix[client_1_idx][client_2_idx] = travel_time
                    except IndexError:
                        matrix[client_1_idx][client_2_idx] = out_value

    return np.array(matrix)

In [8]:
client_time_matrix_car = define_client_time_matrix(c_c_dist, vehicle_type="Car")
client_time_matrix_bike = define_client_time_matrix(c_c_dist, vehicle_type="Bike")

In [9]:
list_inter_columns = c_n_dist.columns[2:]
full_inter_list = list(set([int(x.split(sep="_")[2]) for x in list_inter_columns]))

inter_id_to_idx = {}
inter_idx_to_id = {}
for k in range(len(full_inter_list)):
    inter_id_to_idx[full_inter_list[k]] = k
    inter_idx_to_id[k] = full_inter_list[k]

In [10]:
def define_inter_time_matrix(
    df,
    full_client_list,
    inter_id_to_idx=inter_id_to_idx,
    client_id_to_idx=client_id_to_idx,
    vehicle_type="Car",
    out_value=4000,
):
    list_inter_columns = df.columns[2:]
    full_inter_list = list(set([int(x.split(sep="_")[2]) for x in list_inter_columns]))

    matrix = [
        [0 for i in range(len(full_client_list))] for k in range(len(full_inter_list))
    ]

    for inter_id in full_inter_list:
        vehicle_str = "_Driving" if vehicle_type == "Car" else "_Bicycling"
        col_name = "Duration_Intervenant_" + str(inter_id) + vehicle_str
        inter_idx = inter_id_to_idx[inter_id]

        for client_id in full_client_list:
            client_idx = client_id_to_idx[client_id]
            try:
                travel_time = convert_duration(
                    df.loc[df["ID Client"] == client_id, col_name].values[0]
                )
                matrix[inter_idx][client_idx] = travel_time
            except IndexError:
                matrix[inter_idx][client_idx] = out_value

    return np.array(matrix)

In [11]:
inter_time_matrix_car = define_inter_time_matrix(
    c_n_dist, full_client_list, vehicle_type="Car"
)
inter_time_matrix_bike = define_inter_time_matrix(
    c_n_dist, full_client_list, vehicle_type="Bike"
)

In [12]:
def create_full_time_matrix(client_time_matrix, inter_time_matrix, out_value=4000):
    nb_client = client_time_matrix.shape[0]
    nb_inter = inter_time_matrix.shape[0]

    matrix = [
        [out_value for i in range(nb_client + nb_inter)]
        for k in range(nb_client + nb_inter)
    ]

    for i in range(nb_client):
        for j in range(nb_client):
            matrix[i][j] = client_time_matrix[i][j]

    for i in range(nb_inter):
        for j in range(nb_client):
            matrix[i + nb_client][j] = inter_time_matrix[i][j]
            matrix[j][i + nb_client] = matrix[i + nb_client][j]

    return np.array(matrix)

In [13]:
full_time_matrix_car = create_full_time_matrix(
    client_time_matrix_car, inter_time_matrix_car
)
full_time_matrix_bike = create_full_time_matrix(
    client_time_matrix_bike, inter_time_matrix_bike
)

In [14]:
list_skills = c_l_need["Prestation"].unique()

condition = c_l_need["Date"] == "2024-01-01"
c_l_test = c_l_need[condition]
c_l_test.drop(columns=["Date"], inplace=True)
# c_l_test = c_l_test[:70]
c_l_test.head()

/var/folders/95/2w4zxm7j571dt7gznpftznw80000gn/T/ipykernel_86218/3667956813.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c_l_test.drop(columns=["Date"], inplace=True)


,ID Client,Prestation,Durée,tranche_horaire
0,559475456,REPAS,30.0,"[7, 9]"
1,559277088,TOILETTE,45.0,"[7, 11]"
2,87852633,TOILETTE,45.0,"[7, 11]"
3,243033408,TOILETTE,30.0,"[7, 11]"
4,814940942,TOILETTE,95.0,"[7, 11]"


# 3. Data and function preparation for scenarios


In [15]:
data_path = "ChallengeXHEC23022024.xlsx"
dict_df_sheets = pd.read_excel(data_path, sheet_name=None)
df_hist_jan = dict_df_sheets["JAN24"].copy()
df_clients = dict_df_sheets["clients"].copy()
df_inter = dict_df_sheets["intervenants"].copy()

In [16]:
list_jan_dates = ["2024-01-" + ("0" if k < 10 else "") + str(k) for k in range(1, 32)]
df_inter_columns = [*df_inter.columns[:7], *list_jan_dates]
df_inter.columns = df_inter_columns

In [17]:
client_locations = [
    (df_clients["Latitude"][k], df_clients["Longitude"][k])
    for k in range(len(df_clients))
]

inter_locations = [
    (df_inter["Latitude"][k], df_inter["Longitude"][k]) for k in range(len(df_inter))
]

In [18]:
client_id_to_location = {}
for k in range(len(df_clients)):
    client_id_to_location[int(df_clients["ID Client"][k])] = client_locations[k]

inter_id_to_location = {}
for k in range(len(df_inter)):
    inter_id_to_location[int(df_inter["ID Intervenant"][k])] = inter_locations[k]

In [19]:
# Get the list of Intervenant IDs which are available on the given date
def get_list_og_available(date, df_inter):
    condition = df_inter[date] == 1
    list_available_og = list(df_inter[condition]["ID Intervenant"])
    return list_available_og

# 3.1. Travel time estimator from locations


Necessary to avoid a call to Google Maps API, the estimators output travel times from two locations


In [20]:
from sklearn.ensemble import RandomForestRegressor

In [21]:
# Build the dataset

nb_clients = len(full_client_list)
nb_inter = len(full_inter_list)
nb_loc = len(full_time_matrix_car)

travel_time_data = []

for i in range(nb_loc - 1):
    for j in range(i + 1, nb_loc):
        row = []
        if i < nb_clients:
            client_id_1 = client_idx_to_id[i]
            row.append(client_id_to_location[client_id_1][0])
            row.append(client_id_to_location[client_id_1][1])

            if j < nb_clients:
                client_id_2 = client_idx_to_id[j]
                row.append(client_id_to_location[client_id_2][0])
                row.append(client_id_to_location[client_id_2][1])
            else:
                inter_id_2 = inter_idx_to_id[j - nb_clients]
                row.append(inter_id_to_location[inter_id_2][0])
                row.append(inter_id_to_location[inter_id_2][1])

        else:
            inter_id = inter_idx_to_id[i - nb_clients]
            row.append(inter_id_to_location[inter_id][0])
            row.append(inter_id_to_location[inter_id][1])

            if j < nb_clients:
                client_id_2 = client_idx_to_id[j]
                row.append(client_id_to_location[client_id_2][0])
                row.append(client_id_to_location[client_id_2][1])
            else:
                inter_id_2 = inter_idx_to_id[j - nb_clients]
                row.append(inter_id_to_location[inter_id_2][0])
                row.append(inter_id_to_location[inter_id_2][1])

        row.append(full_time_matrix_car[i][j])
        row.append(full_time_matrix_bike[i][j])

        travel_time_data.append(row)

travel_time_df = pd.DataFrame(
    travel_time_data, columns=["x_1", "y_1", "x_2", "y_2", "car", "bike"]
)
travel_time_df

,x_1,y_1,x_2,y_2,car,bike
0,48.732218,1.363744,48.705077,1.333307,7,14
1,48.732218,1.363744,48.638658,1.517034,24,63
2,48.732218,1.363744,48.787278,1.441202,16,38
3,48.732218,1.363744,48.768307,1.403410,10,24
4,48.732218,1.363744,48.725093,1.427367,11,25
...,...,...,...,...,...,...
9586,48.758099,1.210611,48.689927,1.351054,4000,4000
9587,48.758099,1.210611,48.763226,1.241120,4000,4000
9588,48.729206,1.371985,48.689927,1.351054,4000,4000
9589,48.729206,1.371985,48.763226,1.241120,4000,4000


In [22]:
condition = travel_time_df["car"] != 4000
car_travel_time_df = travel_time_df[condition].copy()
X_car = np.array(car_travel_time_df.drop(columns=["car", "bike"]))
y_car = np.array(car_travel_time_df["car"])

car_time_travel_estimator = RandomForestRegressor()
car_time_travel_estimator.fit(X_car, y_car)

RandomForestRegressor()

In [23]:
condition = travel_time_df["bike"] != 4000
bike_travel_time_df = travel_time_df[condition].copy()
X_bike = np.array(bike_travel_time_df.drop(columns=["car", "bike"]))
y_bike = np.array(bike_travel_time_df["bike"])

bike_time_travel_estimator = RandomForestRegressor()
bike_time_travel_estimator.fit(X_bike, y_bike)

RandomForestRegressor()

## 3.2. Add synthetic clients


The goal is to create artificially a new client the following way:

- Get a random ('Prestation', 'Durée', 'Tranche horaire') from an existing client;
- Get a random location

From the random location, infere the travel time to each other node


In [24]:
# From a given travel time matrix and set of locations, outputs a new travel time matrix
def add_clients_in_time_matrix(
    client_synthetic_locations,
    time_matrix_car,
    time_matrix_bike,
    df_og_need,
    list_available_inter,
):
    nb_og_clients = len(df_og_need)
    len_new_matrix = (
        nb_og_clients + len(client_synthetic_locations) + len(list_available_inter)
    )
    new_car_matrix = [
        [4000 for k in range(len_new_matrix)] for k in range(len_new_matrix)
    ]
    new_bike_matrix = [
        [4000 for k in range(len_new_matrix)] for k in range(len_new_matrix)
    ]

    nb_synth_clients = len(client_synthetic_locations)

    for i in range(len_new_matrix - 1):
        for j in range(i + 1, len_new_matrix):
            if i < nb_og_clients:
                if j < nb_og_clients:
                    new_car_matrix[i][j] = time_matrix_car[i][j]
                    new_bike_matrix[i][j] = time_matrix_bike[i][j]
                elif nb_og_clients <= j < nb_og_clients + nb_synth_clients:
                    # client_id_1 = client_idx_to_id[i]
                    client_id_1 = df_og_need["ID Client"][i]
                    client_loc_1 = client_id_to_location[client_id_1]
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    client_loc_1[0],
                                    client_loc_1[1],
                                    client_synthetic_locations[j - nb_og_clients][0],
                                    client_synthetic_locations[j - nb_og_clients][1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    client_loc_1[0],
                                    client_loc_1[1],
                                    client_synthetic_locations[j - nb_og_clients][0],
                                    client_synthetic_locations[j - nb_og_clients][1],
                                ]
                            ]
                        )[0]
                    )
                else:
                    new_car_matrix[i][j] = time_matrix_car[i][j - nb_synth_clients]
                    new_bike_matrix[i][j] = time_matrix_bike[i][j - nb_synth_clients]
            elif nb_og_clients <= i < nb_og_clients + nb_synth_clients:
                if j < nb_og_clients + nb_synth_clients:
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    client_synthetic_locations[j - nb_og_clients][0],
                                    client_synthetic_locations[j - nb_og_clients][1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    client_synthetic_locations[j - nb_og_clients][0],
                                    client_synthetic_locations[j - nb_og_clients][1],
                                ]
                            ]
                        )[0]
                    )
                else:
                    # inter_id_1 = inter_idx_to_id[j - nb_og_clients - nb_synth_clients]
                    # print(i)
                    # print(j)
                    # print(j - nb_og_clients - nb_synth_clients)
                    # print("\n")
                    inter_id_1 = list_available_inter[
                        j - nb_og_clients - nb_synth_clients
                    ]
                    inter_loc_1 = inter_id_to_location[inter_id_1]
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    inter_loc_1[0],
                                    inter_loc_1[1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    inter_loc_1[0],
                                    inter_loc_1[1],
                                ]
                            ]
                        )[0]
                    )
            else:
                new_car_matrix[i][j] = time_matrix_car[i - nb_synth_clients][
                    j - nb_synth_clients
                ]
                new_bike_matrix[i][j] = time_matrix_bike[i - nb_synth_clients][
                    j - nb_synth_clients
                ]

            new_car_matrix[j][i] = new_car_matrix[i][j]
            new_bike_matrix[j][i] = new_bike_matrix[i][j]

    for i in range(len_new_matrix):
        new_car_matrix[i][i] = 0
        new_bike_matrix[i][i] = 0

    return np.array(new_car_matrix), np.array(new_bike_matrix)

## 3.3. Add synthetic inter


The goal is to create artificially a new inter the following way:

- Get a random ('Skills', 'Disponibility', 'Car') from existing inter (can be different ones);
- Get a random location

From the random location, infere the travel time to each other node


In [25]:
# Same as adding clients
def add_clients_and_inter_in_time_matrix(
    client_synthetic_locations,
    inter_synthetic_locations,
    time_matrix_car,
    time_matrix_bike,
    df_og_need,
    list_available_inter,
):
    nb_og_clients = len(df_og_need)
    nb_og_inter = len(list_available_inter)
    len_new_matrix = (
        len(client_synthetic_locations)
        + len(inter_synthetic_locations)
        + len(time_matrix_car)
    )
    new_car_matrix = [
        [4000 for k in range(len_new_matrix)] for k in range(len_new_matrix)
    ]
    new_bike_matrix = [
        [4000 for k in range(len_new_matrix)] for k in range(len_new_matrix)
    ]

    nb_synth_clients = len(client_synthetic_locations)
    nb_synth_inter = len(inter_synthetic_locations)

    for i in range(len_new_matrix - 1):
        for j in range(i + 1, len_new_matrix):
            if i < nb_og_clients:
                if j < nb_og_clients:
                    new_car_matrix[i][j] = time_matrix_car[i][j]
                    new_bike_matrix[i][j] = time_matrix_bike[i][j]
                elif nb_og_clients <= j < nb_og_clients + nb_synth_clients:
                    client_id_1 = df_og_need["ID Client"][i]
                    client_loc_1 = client_id_to_location[client_id_1]
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    client_loc_1[0],
                                    client_loc_1[1],
                                    client_synthetic_locations[j - nb_og_clients][0],
                                    client_synthetic_locations[j - nb_og_clients][1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    client_loc_1[0],
                                    client_loc_1[1],
                                    client_synthetic_locations[j - nb_og_clients][0],
                                    client_synthetic_locations[j - nb_og_clients][1],
                                ]
                            ]
                        )[0]
                    )
                elif (
                    nb_og_clients + nb_synth_clients
                    <= j
                    < nb_og_clients + nb_synth_clients + nb_og_inter
                ):
                    new_car_matrix[i][j] = time_matrix_car[i][j - nb_synth_clients]
                    new_bike_matrix[i][j] = time_matrix_bike[i][j - nb_synth_clients]
                else:
                    client_id_1 = df_og_need["ID Client"][i]
                    client_loc_1 = client_id_to_location[client_id_1]
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    client_loc_1[0],
                                    client_loc_1[1],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    client_loc_1[0],
                                    client_loc_1[1],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
            elif nb_og_clients <= i < nb_og_clients + nb_synth_clients:
                if j < nb_og_clients + nb_synth_clients:
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    client_synthetic_locations[j - nb_og_clients][0],
                                    client_synthetic_locations[j - nb_og_clients][1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    client_synthetic_locations[j - nb_og_clients][0],
                                    client_synthetic_locations[j - nb_og_clients][1],
                                ]
                            ]
                        )[0]
                    )
                elif (
                    nb_og_clients + nb_synth_clients
                    <= j
                    < nb_og_clients + nb_synth_clients + nb_og_inter
                ):
                    # inter_id_1 = inter_idx_to_id[j - nb_og_clients - nb_synth_clients]
                    inter_id_1 = list_available_inter[
                        j - nb_og_clients - nb_synth_clients
                    ]
                    inter_loc_1 = inter_id_to_location[inter_id_1]
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    inter_loc_1[0],
                                    inter_loc_1[1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    inter_loc_1[0],
                                    inter_loc_1[1],
                                ]
                            ]
                        )[0]
                    )
                else:
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    client_synthetic_locations[i - nb_og_clients][0],
                                    client_synthetic_locations[i - nb_og_clients][1],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
            elif (
                nb_og_clients + nb_synth_clients
                <= i
                < nb_og_clients + nb_synth_clients + nb_og_inter
            ):
                if j < nb_og_clients + nb_synth_clients + nb_og_inter:
                    new_car_matrix[i][j] = time_matrix_car[i - nb_synth_clients][
                        j - nb_synth_clients
                    ]
                    new_bike_matrix[i][j] = time_matrix_bike[i - nb_synth_clients][
                        j - nb_synth_clients
                    ]
                else:
                    # inter_id_1 = inter_idx_to_id[i - nb_synth_clients - nb_og_clients]
                    inter_id_1 = list_available_inter[
                        i - nb_og_clients - nb_synth_clients
                    ]
                    inter_loc_1 = inter_id_to_location[inter_id_1]
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    inter_loc_1[0],
                                    inter_loc_1[1],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    inter_loc_1[0],
                                    inter_loc_1[1],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j
                                        - nb_og_clients
                                        - nb_synth_clients
                                        - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
            else:
                new_car_matrix[i][j] = round(
                    car_time_travel_estimator.predict(
                        [
                            [
                                inter_synthetic_locations[
                                    i - nb_og_clients - nb_synth_clients - nb_og_inter
                                ][0],
                                inter_synthetic_locations[
                                    i - nb_og_clients - nb_synth_clients - nb_og_inter
                                ][1],
                                inter_synthetic_locations[
                                    j - nb_og_clients - nb_synth_clients - nb_og_inter
                                ][0],
                                inter_synthetic_locations[
                                    j - nb_og_clients - nb_synth_clients - nb_og_inter
                                ][1],
                            ]
                        ]
                    )[0]
                )
                new_bike_matrix[i][j] = round(
                    bike_time_travel_estimator.predict(
                        [
                            [
                                inter_synthetic_locations[
                                    i - nb_og_clients - nb_synth_clients - nb_og_inter
                                ][0],
                                inter_synthetic_locations[
                                    i - nb_og_clients - nb_synth_clients - nb_og_inter
                                ][1],
                                inter_synthetic_locations[
                                    j - nb_og_clients - nb_synth_clients - nb_og_inter
                                ][0],
                                inter_synthetic_locations[
                                    j - nb_og_clients - nb_synth_clients - nb_og_inter
                                ][1],
                            ]
                        ]
                    )[0]
                )

            new_car_matrix[j][i] = new_car_matrix[i][j]
            new_bike_matrix[j][i] = new_bike_matrix[i][j]

    for i in range(len_new_matrix):
        new_car_matrix[i][i] = 0
        new_bike_matrix[i][i] = 0

    return np.array(new_car_matrix), np.array(new_bike_matrix)

In [26]:
# Same as adding clients
def add_inter_in_time_matrix(
    inter_synthetic_locations,
    time_matrix_car,
    time_matrix_bike,
    df_og_need,
    list_available_inter,
):
    nb_og_clients = len(df_og_need)
    nb_og_inter = len(list_available_inter)
    len_new_matrix = +len(inter_synthetic_locations) + len(time_matrix_car)
    new_car_matrix = [
        [4000 for k in range(len_new_matrix)] for k in range(len_new_matrix)
    ]
    new_bike_matrix = [
        [4000 for k in range(len_new_matrix)] for k in range(len_new_matrix)
    ]

    for i in range(len_new_matrix - 1):
        for j in range(i + 1, len_new_matrix):
            if i < nb_og_clients:
                if j < nb_og_clients:
                    new_car_matrix[i][j] = time_matrix_car[i][j]
                    new_bike_matrix[i][j] = time_matrix_bike[i][j]
                elif nb_og_clients <= j < nb_og_clients + nb_og_inter:
                    new_car_matrix[i][j] = time_matrix_car[i][j]
                    new_bike_matrix[i][j] = time_matrix_bike[i][j]
                else:
                    # client_id_1 = client_idx_to_id[i]
                    client_id_1 = df_og_need["ID Client"][i]
                    client_loc_1 = client_id_to_location[client_id_1]
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    client_loc_1[0],
                                    client_loc_1[1],
                                    inter_synthetic_locations[
                                        j - nb_og_clients - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j - nb_og_clients - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    client_loc_1[0],
                                    client_loc_1[1],
                                    inter_synthetic_locations[
                                        j - nb_og_clients - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j - nb_og_clients - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
            elif nb_og_clients <= i < nb_og_clients + nb_og_inter:
                if j < nb_og_clients + nb_og_inter:
                    new_car_matrix[i][j] = time_matrix_car[i][j]
                    new_bike_matrix[i][j] = time_matrix_bike[i][j]
                else:
                    # inter_id_1 = inter_idx_to_id[i - nb_og_clients]
                    inter_id_1 = list_available_inter[i - nb_og_clients]
                    inter_loc_1 = inter_id_to_location[inter_id_1]
                    new_car_matrix[i][j] = round(
                        car_time_travel_estimator.predict(
                            [
                                [
                                    inter_loc_1[0],
                                    inter_loc_1[1],
                                    inter_synthetic_locations[
                                        j - nb_og_clients - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j - nb_og_clients - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
                    new_bike_matrix[i][j] = round(
                        bike_time_travel_estimator.predict(
                            [
                                [
                                    inter_loc_1[0],
                                    inter_loc_1[1],
                                    inter_synthetic_locations[
                                        j - nb_og_clients - nb_og_inter
                                    ][0],
                                    inter_synthetic_locations[
                                        j - nb_og_clients - nb_og_inter
                                    ][1],
                                ]
                            ]
                        )[0]
                    )
            else:
                new_car_matrix[i][j] = round(
                    car_time_travel_estimator.predict(
                        [
                            [
                                inter_synthetic_locations[
                                    i - nb_og_clients - nb_og_inter
                                ][0],
                                inter_synthetic_locations[
                                    i - nb_og_clients - nb_og_inter
                                ][1],
                                inter_synthetic_locations[
                                    j - nb_og_clients - nb_og_inter
                                ][0],
                                inter_synthetic_locations[
                                    j - nb_og_clients - nb_og_inter
                                ][1],
                            ]
                        ]
                    )[0]
                )
                new_bike_matrix[i][j] = round(
                    bike_time_travel_estimator.predict(
                        [
                            [
                                inter_synthetic_locations[
                                    i - nb_og_clients - nb_og_inter
                                ][0],
                                inter_synthetic_locations[
                                    i - nb_og_clients - nb_og_inter
                                ][1],
                                inter_synthetic_locations[
                                    j - nb_og_clients - nb_og_inter
                                ][0],
                                inter_synthetic_locations[
                                    j - nb_og_clients - nb_og_inter
                                ][1],
                            ]
                        ]
                    )[0]
                )

            new_car_matrix[j][i] = new_car_matrix[i][j]
            new_bike_matrix[j][i] = new_bike_matrix[i][j]

    for i in range(len_new_matrix):
        new_car_matrix[i][i] = 0
        new_bike_matrix[i][i] = 0

    return np.array(new_car_matrix), np.array(new_bike_matrix)

## 3.4. Create synthetic actors (clients and intervenants)


In [27]:
def sample_random_loc(nb_synth_loc, list_og_locations):
    og_x_coords, og_y_coords = zip(*list_og_locations)

    # Calculate mean and standard deviation for x and y
    og_mean_x, og_std_x = np.mean(og_x_coords), np.std(og_x_coords)
    og_mean_y, og_std_y = np.mean(og_y_coords), np.std(og_y_coords)

    # Generate synthetic locations
    list_synth_locations = [
        (
            np.random.normal(og_mean_x, og_std_x),
            np.random.normal(og_mean_y, og_std_y),
        )
        for _ in range(nb_synth_loc)
    ]

    return list_synth_locations

In [28]:
# From a baseline, create new actors and add them to the baseline
def add_actors(
    df_og_needs,
    df_og_inter,
    list_available_inter,
    time_matrix_car,
    time_matrix_bike,
    nb_synth_clients=0,
    nb_synth_inter=0,
    client_locations=client_locations,
    inter_locations=inter_locations,
    inter_synth_constraints=False,
    date=None,
):
    if nb_synth_inter == 0:
        list_synth_client_loc = sample_random_loc(nb_synth_clients, client_locations)
        new_car_time_matrix, new_bike_time_matrix = add_clients_in_time_matrix(
            list_synth_client_loc,
            time_matrix_car,
            time_matrix_bike,
            df_og_needs,
            list_available_inter,
        )

        synth_client_needs_df = df_og_needs[
            ["Prestation", "Durée", "tranche_horaire"]
        ].sample(n=nb_synth_clients, replace=True)
        synth_client_needs_df.reset_index(drop=True, inplace=True)

        synth_available_inter_df = pd.DataFrame()

    elif nb_synth_clients == 0:
        synth_client_needs_df = pd.DataFrame()

        synth_inter_df = df_og_inter.sample(n=nb_synth_inter, replace=True)
        synth_inter_df.reset_index(drop=True, inplace=True)

        list_synth_inter_loc = sample_random_loc(nb_synth_inter, inter_locations)
        synth_inter_df["Latitude"] = [x[0] for x in list_synth_inter_loc]
        synth_inter_df["Longitude"] = [x[1] for x in list_synth_inter_loc]

        if inter_synth_constraints:
            condition = synth_inter_df[date] == 1
            synth_available_inter_df = synth_inter_df[condition].copy()
            synth_available_inter_df.reset_index(drop=True, inplace=True)
            nb_synth_inter = len(synth_available_inter_df)
            list_synth_available_inter_loc = [
                (
                    synth_available_inter_df["Latitude"][k],
                    synth_available_inter_df["Longitude"][k],
                )
                for k in range(len(synth_available_inter_df))
            ]
        else:
            synth_available_inter_df = synth_inter_df
            list_synth_available_inter_loc = list_synth_inter_loc

        new_car_time_matrix, new_bike_time_matrix = add_inter_in_time_matrix(
            list_synth_available_inter_loc,
            time_matrix_car,
            time_matrix_bike,
            df_og_needs,
            list_available_inter,
        )

    else:
        list_synth_client_loc = sample_random_loc(nb_synth_clients, client_locations)

        synth_client_needs_df = df_og_needs[
            ["Prestation", "Durée", "tranche_horaire"]
        ].sample(n=nb_synth_clients, replace=True)
        synth_client_needs_df.reset_index(drop=True, inplace=True)

        synth_inter_df = df_og_inter.sample(n=nb_synth_inter, replace=True)
        synth_inter_df.reset_index(drop=True, inplace=True)

        list_synth_inter_loc = sample_random_loc(nb_synth_inter, inter_locations)
        synth_inter_df["Latitude"] = [x[0] for x in list_synth_inter_loc]
        synth_inter_df["Longitude"] = [x[1] for x in list_synth_inter_loc]

        if inter_synth_constraints:
            condition = synth_inter_df[date] == 1
            synth_available_inter_df = synth_inter_df[condition].copy()
            synth_available_inter_df.reset_index(drop=True, inplace=True)
            nb_synth_inter = len(synth_available_inter_df)
            list_synth_available_inter_loc = [
                (
                    synth_available_inter_df["Latitude"][k],
                    synth_available_inter_df["Longitude"][k],
                )
                for k in range(len(synth_available_inter_df))
            ]
        else:
            synth_available_inter_df = synth_inter_df
            list_synth_available_inter_loc = list_synth_inter_loc

        new_car_time_matrix, new_bike_time_matrix = (
            add_clients_and_inter_in_time_matrix(
                list_synth_client_loc,
                list_synth_available_inter_loc,
                time_matrix_car,
                time_matrix_bike,
                df_og_needs,
                list_available_inter,
            )
        )

    return (
        new_car_time_matrix,
        new_bike_time_matrix,
        synth_client_needs_df,
        synth_available_inter_df,
    )

In [29]:
# Modify the model's data creation function to incorporate the possibility of having synthetic actors (see last modification below for arguments definition)
def create_real_data_model_with_synth(
    df_og_need,
    full_time_matrix_car,
    full_time_matrix_bike,
    list_available_inter,
    nb_synth_clients=0,
    nb_synth_inter=0,
    client_id_to_idx=client_id_to_idx,
    inter_id_to_idx=inter_id_to_idx,
    nb_full_client=nb_full_client,
    list_skills=list_skills,
    df_inter=df_inter,
):
    nb_needs = len(df_og_need)
    nb_inter = len(list_available_inter)
    data = {}

    # Initialization
    data["og_time_matrix_car"] = [
        [4000 for i in range(nb_needs + nb_inter)] for k in range(nb_needs + nb_inter)
    ]
    data["og_time_matrix_bike"] = [
        [4000 for i in range(nb_needs + nb_inter)] for k in range(nb_needs + nb_inter)
    ]
    data["service_times"] = [
        0 for i in range(nb_needs + nb_inter + nb_synth_clients + nb_synth_inter)
    ]
    data["time_windows"] = [
        (0, 1440)
        for i in range(nb_needs + nb_inter + nb_synth_clients + nb_synth_inter)
    ]

    data["num_vehicles"] = nb_inter + nb_synth_inter
    data["is_car"] = [
        1 for i in range(nb_inter + nb_synth_inter)
    ]  # For the moment every inter has a car
    data["starts"] = [
        nb_needs + nb_synth_clients + i for i in range(nb_inter + nb_synth_inter)
    ]
    data["ends"] = [
        nb_needs + nb_synth_clients + i for i in range(nb_inter + nb_synth_inter)
    ]

    data["vehicle_skills"] = [
        list_skills for i in range(nb_inter + nb_synth_inter)
    ]  # For the moment every inter has all skills possible
    data["node_requirements"] = [
        None for i in range(nb_needs + nb_inter + nb_synth_clients + nb_synth_inter)
    ]

    # Create original time matrix
    for i in range(nb_needs):
        need_id_client = df_og_need["ID Client"][i]
        need_idx_client = client_id_to_idx[need_id_client]

        for j in range(nb_needs):
            other_id_client = df_og_need["ID Client"][j]
            other_idx_client = client_id_to_idx[other_id_client]

            travel_time_car = full_time_matrix_car[need_idx_client, other_idx_client]
            travel_time_bike = full_time_matrix_bike[need_idx_client, other_idx_client]
            data["og_time_matrix_car"][i][j] = travel_time_car
            data["og_time_matrix_bike"][i][j] = travel_time_bike

        for j in range(nb_inter):
            id_inter = list_available_inter[j]
            idx_inter = inter_id_to_idx[id_inter]

            travel_time_car = full_time_matrix_car[need_idx_client][
                nb_full_client + idx_inter
            ]
            travel_time_bike = full_time_matrix_bike[need_idx_client][
                nb_full_client + idx_inter
            ]

            data["og_time_matrix_car"][i][nb_needs + j] = travel_time_car
            data["og_time_matrix_car"][nb_needs + j][i] = data["og_time_matrix_car"][i][
                nb_needs + j
            ]
            data["og_time_matrix_bike"][i][nb_needs + j] = travel_time_bike
            data["og_time_matrix_bike"][nb_needs + j][i] = data["og_time_matrix_bike"][
                i
            ][nb_needs + j]
    for i in range(nb_needs + nb_inter):
        data["og_time_matrix_car"][i][i] = 0
        data["og_time_matrix_bike"][i][i] = 0

    # Create new time matrix from orginial adding synthetic travel times
    (
        data["time_matrix_car"],
        data["time_matrix_bike"],
        synth_client_needs_df,
        synth_inter_df,
    ) = add_actors(
        df_og_need,
        df_inter,
        list_available_inter,
        data["og_time_matrix_car"],
        data["og_time_matrix_bike"],
        nb_synth_clients,
        nb_synth_inter,
        client_locations,
        inter_locations,
        inter_synth_constraints=int,
    )

    for i in range(nb_needs + nb_synth_clients):
        if i < nb_needs:
            service_time = int(df_og_need["Durée"][i])
            data["service_times"][i] = service_time

            time_window_temp = (
                df_og_need["tranche_horaire"][i]
                .replace("[", "")
                .replace("]", "")
                .replace(",", "")
                .split()
            )
            time_window_start = int(float(time_window_temp[0]) * 60)
            time_window_end = int(float(time_window_temp[1]) * 60)
            data["time_windows"][i] = (time_window_start, time_window_end)

            skills_required = df_og_need["Prestation"][i]
            data["node_requirements"][i] = skills_required
        else:
            service_time = int(synth_client_needs_df["Durée"][i - nb_needs])
            data["service_times"][i] = service_time

            time_window_temp = (
                synth_client_needs_df["tranche_horaire"][i - nb_needs]
                .replace("[", "")
                .replace("]", "")
                .replace(",", "")
                .split()
            )
            time_window_start = int(float(time_window_temp[0]) * 60)
            time_window_end = int(float(time_window_temp[1]) * 60)
            data["time_windows"][i] = (time_window_start, time_window_end)

            skills_required = synth_client_needs_df["Prestation"][i - nb_needs]
            data["node_requirements"][i] = skills_required

    data["time_windows_with_service"] = [
        (
            data["time_windows"][i][0] + data["service_times"][i],
            data["time_windows"][i][1] + data["service_times"][i],
        )
        for i in range(len(data["service_times"]))
    ]

    data["workload_per_node"] = [1 for i in range(nb_needs + nb_synth_clients)].append(
        [0 for k in range(nb_inter + nb_synth_inter)]
    )  # Useless for now but could be used to differ workload from each node
    data["workload_capacity"] = [5 for i in range(nb_inter + nb_synth_inter)]

    return data

## 3.5. Refined printing output function


In [30]:
def eq_co2_car(travel_time):
    # 10km = 1.9kg CO2
    speed = 0.5  # km/min
    return 1.9 * travel_time * speed / 10


def eq_co2_elec_bike(travel_time):
    # 10km = 20g CO2
    speed = 1 / 3  # km/min
    return 0.02 * travel_time * speed / 10

In [31]:
# Modify the model's output function to incorporate the possibility of having synthetic actors, the kgCO2 equivalent and !!!the following subparts of part 6
def print_detailed_solution_real_data(
    data, manager, routing, solution, list_available_inter, verbose=0
):
    """Print the details of the model's solution

    Args:
        data (dict): data dict created and used by the model
        manager (_type_): see Google OR Tools
        routing (_type_): see Google OR Tools
        solution (_type_): see Google OR Tools
        list_available_inter (list(int)): list of original (not synthetic) inter IDs
        verbose (int, optional): 0: prints overall results for the day; 1: print detailed routes. Defaults to 0.

    Returns:
        dict: dict of results
    """
    time_dimension = routing.GetDimensionOrDie("Time")

    nb_cars_used = 0
    nb_bikes_used = 0
    nb_inter_used = 0

    nb_break_inf = 0
    nb_break_sup = 0
    nb_visits = 0

    avg_length_day = 0
    list_length_day = []

    overall_total_car_commuting_time = 0
    overall_total_bike_commuting_time = 0
    overall_total_commuting_time = 0  # Initialize overall commuting time

    # Display dropped nodes.
    dropped_nodes = "Dropped nodes:"
    count_dropped_nodes = 0
    for node in range(routing.Size()):
        if routing.IsStart(node) or routing.IsEnd(node):
            continue
        if solution.Value(routing.NextVar(node)) == node:
            dropped_nodes += f" {manager.IndexToNode(node)};"
            count_dropped_nodes += 1
    print(f"{dropped_nodes} (Total: {count_dropped_nodes})")

    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        vehicle_skills = str(data["vehicle_skills"][vehicle_id])
        try:
            inter_id = list_available_inter[vehicle_id]
        except IndexError:
            inter_id = "*synthetic*"
        vehicle_type_str = "Car" if data["is_car"][vehicle_id] else "Bike"
        if verbose == 1:
            print(
                f"Route for vehicle {inter_id} (Type: {vehicle_type_str}; Skills: {vehicle_skills}):"
            )

        vehicle_commuting_time = 0  # Initialize commuting time for the current vehicle
        nb_steps = 0
        start_day = 0
        end_day = 0
        nb_break_inf_veh = 0
        nb_break_sup_veh = 0

        # print(f"Route for vehicle {inter_id}:")
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            next_index = solution.Value(routing.NextVar(index))
            next_node = manager.IndexToNode(next_index)

            if nb_steps == 1:
                if data["is_car"][vehicle_id]:
                    nb_cars_used += 1
                else:
                    nb_bikes_used += 1
                nb_inter_used += 1

            service_time = data["service_times"][manager.IndexToNode(index)]
            skills_required = data["node_requirements"][manager.IndexToNode(index)]
            # Arrival time at current location
            arrival_time = solution.Min(time_var) - service_time

            # End of service time = arrival time + service time at current location
            end_of_service_time = solution.Min(time_var)

            # Initialization
            departure_time = end_of_service_time

            if data["is_car"][vehicle_id]:
                next_travel_time = data["time_matrix_car"][manager.IndexToNode(index)][
                    next_node
                ]
            else:
                next_travel_time = data["time_matrix_bike"][manager.IndexToNode(index)][
                    next_node
                ]
            vehicle_commuting_time += next_travel_time
            if data["is_car"][vehicle_id]:
                overall_total_car_commuting_time += next_travel_time
            else:
                overall_total_bike_commuting_time += next_travel_time
            overall_total_commuting_time += next_travel_time

            if next_index != routing.End(vehicle_id):
                next_end_of_service_time = solution.Min(
                    time_dimension.CumulVar(next_index)
                )
                next_service_time = data["service_times"][next_node]
                next_arrival_time = next_end_of_service_time - next_service_time
                departure_time = next_arrival_time - next_travel_time

            break_time = departure_time - end_of_service_time

            if nb_steps > 0:
                if break_time > 30:
                    nb_break_sup += 1
                    nb_break_sup_veh += 1
                    nb_visits += 1
                else:
                    nb_visits += 1
                    if next_index != routing.End(vehicle_id):
                        nb_break_inf += 1
                        nb_break_inf_veh += 1

            if nb_steps == 0:
                start_day = departure_time

            nb_steps += 1

            if next_index == routing.End(vehicle_id):
                break_time = 0
                departure_time = end_of_service_time

            arrival_time_str = (
                str(arrival_time // 60)
                + "h"
                + (
                    str(arrival_time % 60)
                    if arrival_time % 60 >= 10
                    else "0" + str(arrival_time % 60)
                )
            )
            end_of_service_time_str = (
                str(end_of_service_time // 60)
                + "h"
                + (
                    str(end_of_service_time % 60)
                    if end_of_service_time % 60 >= 10
                    else "0" + str(end_of_service_time % 60)
                )
            )
            break_time_str = (
                str(break_time // 60)
                + "h"
                + (
                    str(break_time % 60)
                    if break_time % 60 >= 10
                    else "0" + str(break_time % 60)
                )
            )
            departure_time_str = (
                str(departure_time // 60)
                + "h"
                + (
                    str(departure_time % 60)
                    if departure_time % 60 >= 10
                    else "0" + str(departure_time % 60)
                )
            )

            if verbose == 1:
                # Print detailed timing for current location
                print(
                    f"* Location {manager.IndexToNode(index)} (Arrival: {arrival_time_str}, "
                    f"End of Service Time (Service time: {service_time}) (Skills required: {skills_required}): {end_of_service_time_str}, Break: {break_time_str}, "
                    f"Departure: {departure_time_str}, Next travel time: {next_travel_time}) -> "
                )

            index = next_index

        # Handle final location's arrival time
        final_arrival_time = solution.Min(time_dimension.CumulVar(index))
        end_day = final_arrival_time
        final_arrival_time_str = (
            str(final_arrival_time // 60)
            + "h"
            + (
                str(final_arrival_time % 60)
                if final_arrival_time % 60 >= 10
                else "0" + str(final_arrival_time % 60)
            )
        )
        if verbose == 1:
            print(
                f"* Location {manager.IndexToNode(index) + 1} (Arrival: {final_arrival_time_str})"
            )
            print(
                f"Number of small breaks (<30min): {nb_break_inf_veh}; Number of big breaks (>30min): {nb_break_sup_veh}"
            )
            print(
                f"Total commuting time for vehicle {inter_id}: {vehicle_commuting_time} minutes"
            )

        if end_day - start_day > 1:  # Superior than 1 minute
            list_length_day.append((start_day, end_day))
            avg_length_day += end_day - start_day

            start_day_str = (
                str(start_day // 60)
                + "h"
                + (
                    str(start_day % 60)
                    if start_day % 60 >= 10
                    else "0" + str(start_day % 60)
                )
            )
            end_day_str = (
                str(end_day // 60)
                + "h"
                + (str(end_day % 60) if end_day % 60 >= 10 else "0" + str(end_day % 60))
            )

            if verbose == 1:
                print(
                    f"Start of the day: {start_day_str}; End of the day: {end_day_str}"
                )
        if verbose == 1:
            print("\n")

    nb_available_inter = data["num_vehicles"]
    print(f"Total available inter: {nb_available_inter}")
    print(f"Total inter who worked: {nb_inter_used}")
    print(f"Toal number of visited nodes: {nb_visits}")
    print(f"Overall total commuting time: {overall_total_commuting_time} minutes")
    print(f"Number of cars used: {nb_cars_used}")
    eq_co2_cars = eq_co2_car(overall_total_car_commuting_time)
    print(
        f"Overall total commuting time for cars: {overall_total_car_commuting_time} minutes ({eq_co2_cars} kgCO2)"
    )
    print(f"Number of bikes used: {nb_bikes_used}")
    eq_co2_bikes = eq_co2_elec_bike(overall_total_bike_commuting_time)
    print(
        f"Overall total commuting time for bikes: {overall_total_bike_commuting_time} minutes ({eq_co2_bikes} kgCO2)"
    )
    print(
        f"Total number of small breaks (<30min): {nb_break_inf}; Total number of big breaks (>30min): {nb_break_sup}"
    )

    avg_length_day = avg_length_day / nb_inter_used
    avg_length_day_str = round(avg_length_day)
    avg_length_day_str = (
        str(avg_length_day_str // 60)
        + "h"
        + (
            str(avg_length_day_str % 60)
            if avg_length_day_str % 60 >= 10
            else "0" + str(avg_length_day_str % 60)
        )
    )
    print(f"Average length of a day of work: {avg_length_day_str}")

    dict_results = {
        "total_commuting_time": overall_total_commuting_time,
        "total_car_commuting_time": overall_total_car_commuting_time,
        "total_bike_commuting_time": overall_total_bike_commuting_time,
        "nb_available_inter": nb_available_inter,
        "nb_inter_used": nb_inter_used,
        "nb_cars_used": nb_cars_used,
        "nb_bikes_used": nb_bikes_used,
        "avg_length_day": avg_length_day,
        "list_length_day": list_length_day,
        "eq_co2_cars": eq_co2_cars,
        "eq_co2_bikes": eq_co2_bikes,
        "nb_break_inf": nb_break_inf,
        "nb_break_sup": nb_break_sup,
        "nb_visits": nb_visits,
        "nb_dropped_nodes": count_dropped_nodes,
    }

    return dict_results

## 3.6. Bikes fleet proportion


In [32]:
def get_is_car(list_available_inter, df_inter):
    list_is_car = []
    for id_inter in list_available_inter:
        condition = df_inter["ID Intervenant"] == id_inter
        if df_inter.loc[condition, "Véhicule personnel"].values[0] == "Oui":
            list_is_car.append(1)
        else:
            list_is_car.append(0)

    return list_is_car

In [33]:
def get_synth_is_car(df_synth_inter):
    list_is_car = []
    for car in df_synth_inter["Véhicule personnel"]:
        if car == "Oui":
            list_is_car.append(1)
        else:
            list_is_car.append(0)

    return list_is_car

In [35]:
def generate_vehicle_type_list(nb_inter, bike_proportion):
    nb_bikes = int(nb_inter * bike_proportion)
    vehicle_type_list = [0] * nb_bikes + [1] * (nb_inter - nb_bikes)
    random.shuffle(vehicle_type_list)
    return vehicle_type_list

## 3.7 Inter constraints


In [36]:
def get_inter_skills(list_available_inter, df_inter):
    list_all_inter_skills = []
    for id_inter in list_available_inter:
        condition = df_inter["ID Intervenant"] == id_inter
        tmp_skills = df_inter.loc[condition, "Compétences"].values[0]
        list_skills = [x.strip() for x in tmp_skills.split(sep=",")]
        list_all_inter_skills.append(list_skills)

    return list_all_inter_skills

In [37]:
def get_synth_inter_skills(synth_inter_df):
    list_all_inter_skills = []
    for tmp_skills in synth_inter_df["Compétences"]:
        list_skills = [x.strip() for x in tmp_skills.split(sep=",")]
        list_all_inter_skills.append(list_skills)

    return list_all_inter_skills

In [38]:
# Modify the model's data creation function to incorporate the possibility of constraints for intervenants (see last modification below for arguments definition)
def create_real_data_model_with_synth(
    df_og_need,
    full_time_matrix_car,
    full_time_matrix_bike,
    list_available_inter,
    nb_synth_clients=0,
    nb_synth_inter=0,
    client_id_to_idx=client_id_to_idx,
    inter_id_to_idx=inter_id_to_idx,
    nb_full_client=nb_full_client,
    list_skills=list_skills,
    df_inter=df_inter,
    inter_constraints=False,
    date=None,
    bike_proportion=0.0,
    workload_capacity=7,
):
    nb_needs = len(df_og_need)
    nb_inter = len(list_available_inter)
    data = {}

    # Initialization
    data["og_time_matrix_car"] = [
        [4000 for i in range(nb_needs + nb_inter)] for k in range(nb_needs + nb_inter)
    ]
    data["og_time_matrix_bike"] = [
        [4000 for i in range(nb_needs + nb_inter)] for k in range(nb_needs + nb_inter)
    ]

    # Create original time matrix
    for i in range(nb_needs):
        need_id_client = df_og_need["ID Client"][i]
        need_idx_client = client_id_to_idx[need_id_client]

        for j in range(nb_needs):
            other_id_client = df_og_need["ID Client"][j]
            other_idx_client = client_id_to_idx[other_id_client]

            travel_time_car = full_time_matrix_car[need_idx_client, other_idx_client]
            travel_time_bike = full_time_matrix_bike[need_idx_client, other_idx_client]
            data["og_time_matrix_car"][i][j] = travel_time_car
            data["og_time_matrix_bike"][i][j] = travel_time_bike

        for j in range(nb_inter):
            id_inter = list_available_inter[j]
            idx_inter = inter_id_to_idx[id_inter]

            travel_time_car = full_time_matrix_car[need_idx_client][
                nb_full_client + idx_inter
            ]
            travel_time_bike = full_time_matrix_bike[need_idx_client][
                nb_full_client + idx_inter
            ]

            data["og_time_matrix_car"][i][nb_needs + j] = travel_time_car
            data["og_time_matrix_car"][nb_needs + j][i] = data["og_time_matrix_car"][i][
                nb_needs + j
            ]
            data["og_time_matrix_bike"][i][nb_needs + j] = travel_time_bike
            data["og_time_matrix_bike"][nb_needs + j][i] = data["og_time_matrix_bike"][
                i
            ][nb_needs + j]
    for i in range(nb_needs + nb_inter):
        data["og_time_matrix_car"][i][i] = 0
        data["og_time_matrix_bike"][i][i] = 0

    # Create new time matrix from orginial adding synthetic travel times
    (
        data["time_matrix_car"],
        data["time_matrix_bike"],
        synth_client_needs_df,
        synth_available_inter_df,
    ) = add_actors(
        df_og_need,
        df_inter,
        list_available_inter,
        data["og_time_matrix_car"],
        data["og_time_matrix_bike"],
        nb_synth_clients,
        nb_synth_inter,
        client_locations,
        inter_locations,
        inter_synth_constraints=inter_constraints,
        date=date,
    )

    if inter_constraints:
        nb_synth_inter = len(synth_available_inter_df)

    data["service_times"] = [
        0 for i in range(nb_needs + nb_inter + nb_synth_clients + nb_synth_inter)
    ]
    data["time_windows"] = [
        (0, 1440)
        for i in range(nb_needs + nb_inter + nb_synth_clients + nb_synth_inter)
    ]

    data["num_vehicles"] = nb_inter + nb_synth_inter

    if inter_constraints:
        list_is_car = get_is_car(list_available_inter, df_inter)
        if nb_synth_inter > 0:
            list_is_car = [*list_is_car, *get_synth_is_car(synth_available_inter_df)]
        data["is_car"] = list_is_car
    else:
        data["is_car"] = [1 for i in range(nb_inter + nb_synth_inter)]
    data["starts"] = [
        nb_needs + nb_synth_clients + i for i in range(nb_inter + nb_synth_inter)
    ]
    data["ends"] = [
        nb_needs + nb_synth_clients + i for i in range(nb_inter + nb_synth_inter)
    ]

    if inter_constraints:
        list_inter_skills = get_inter_skills(list_available_inter, df_inter)
        if nb_synth_inter > 0:
            list_inter_skills = [
                *list_inter_skills,
                *get_synth_inter_skills(synth_available_inter_df),
            ]
        data["vehicle_skills"] = list_inter_skills
    else:
        data["vehicle_skills"] = [list_skills for i in range(nb_inter + nb_synth_inter)]
    data["node_requirements"] = [
        None for i in range(nb_needs + nb_inter + nb_synth_clients + nb_synth_inter)
    ]

    for i in range(nb_needs + nb_synth_clients):
        if i < nb_needs:
            service_time = int(df_og_need["Durée"][i])
            data["service_times"][i] = service_time

            time_window_temp = (
                df_og_need["tranche_horaire"][i]
                .replace("[", "")
                .replace("]", "")
                .replace(",", "")
                .split()
            )
            time_window_start = int(float(time_window_temp[0]) * 60)
            time_window_end = int(float(time_window_temp[1]) * 60)
            data["time_windows"][i] = (time_window_start, time_window_end)

            skills_required = df_og_need["Prestation"][i]
            data["node_requirements"][i] = skills_required
        else:
            service_time = int(synth_client_needs_df["Durée"][i - nb_needs])
            data["service_times"][i] = service_time

            time_window_temp = (
                synth_client_needs_df["tranche_horaire"][i - nb_needs]
                .replace("[", "")
                .replace("]", "")
                .replace(",", "")
                .split()
            )
            time_window_start = int(float(time_window_temp[0]) * 60)
            time_window_end = int(float(time_window_temp[1]) * 60)
            data["time_windows"][i] = (time_window_start, time_window_end)

            skills_required = synth_client_needs_df["Prestation"][i - nb_needs]
            data["node_requirements"][i] = skills_required

    data["time_windows_with_service"] = [
        (
            data["time_windows"][i][0] + data["service_times"][i],
            data["time_windows"][i][1] + data["service_times"][i],
        )
        for i in range(len(data["service_times"]))
    ]

    data["workload_per_node"] = [
        *[1 for i in range(nb_needs + nb_synth_clients)],
        *[0 for k in range(nb_inter + nb_synth_inter)],
    ]  # Useless for now but could be used to differ workload from each node
    data["workload_capacity"] = [
        workload_capacity for i in range(nb_inter + nb_synth_inter)
    ]

    if bike_proportion > 0.00001:
        data["is_car"] = generate_vehicle_type_list(
            nb_synth_inter + nb_inter, bike_proportion
        )

    return data

## 3.8. Training policies


In [39]:
def add_mandatory_skill(skill, list_inter_skills):
    adjusted_list_inter_skills = []
    for list_skills_per_inter in list_inter_skills:
        if skill not in list_skills_per_inter:
            list_skills_per_inter.append(skill)
        adjusted_list_inter_skills.append(list_skills_per_inter)
    return adjusted_list_inter_skills


list_opt_skills = [
    "VIE SOCIALE",
    "AIDE MENAGERE",
    "ACCOMPAGNEMENTS COURSES",
    "FEMME DE MENAGE",
    "GARDE D'ENFANTS",
]


# For each optionnal skill, add it to an inter lacking it with a probability p_skill
def add_proba_skills(list_inter_skills, p_skill, list_opt_skills=list_opt_skills):
    adjusted_list_inter_skills = []
    for list_skills_per_inter in list_inter_skills:
        for opt_skill in list_opt_skills:
            if opt_skill not in list_skills_per_inter:
                p = random.uniform(0, 1)
                if p < p_skill:
                    list_skills_per_inter.append(opt_skill)
        adjusted_list_inter_skills.append(list_skills_per_inter)
    return adjusted_list_inter_skills

In [40]:
# Modify the model's data creation function to incorporate the possibility of training policies
def create_real_data_model_with_synth(
    df_og_need,
    full_time_matrix_car,
    full_time_matrix_bike,
    list_available_inter,
    nb_synth_clients=0,
    nb_synth_inter=0,
    client_id_to_idx=client_id_to_idx,
    inter_id_to_idx=inter_id_to_idx,
    nb_full_client=nb_full_client,
    list_skills=list_skills,
    df_inter=df_inter,
    inter_constraints=False,
    date=None,
    bike_proportion=0.0,
    workload_capacity=7,
    p_training_skills=-1,
):
    """Creates the data dictionnary that is used by the model.

    Args:
        df_og_need (pandas.DataFrame): dataframe of original (and not synthetic) clients needs for a given date
        full_time_matrix_car (2D array): original mapping of travel time by car between all clients and inter
        full_time_matrix_bike (2D array): original mapping of travel time by bike between all clients and inter
        list_available_inter (list(int)): list of original (and not synthetic) inter available for the given date
        nb_synth_clients (int, optional): Number of synthetic clients to add. Defaults to 0.
        nb_synth_inter (int, optional): Number of synthetic inter to add. Defaults to 0.
        client_id_to_idx (dict, optional): Dict mapping original client IDs to their index (for full_time_matrix). Defaults to client_id_to_idx.
        inter_id_to_idx (dict, optional): Dict mapping original inter IDs to their index (for full_time_matrix). Defaults to inter_id_to_idx.
        nb_full_client (int, optional): Number of registered original clients (for full_time_matrix). Defaults to nb_full_client.
        list_skills (list, optional): list of skills by default for each inter (inter_constraints==False). Defaults to list_skills.
        df_inter (pandas.DataFrame, optional): dataframe of original (and not synthetic) inter. Defaults to df_inter.
        inter_constraints (bool, optional): Search for each inter constraints (skills, vehicle types and availability) or not (default). Defaults to False.
        date (String, optional): Date of the optimization, format: "YYYY-MM-DD". Defaults to None.
        bike_proportion (float, optional): Set proportion of available bikes. Defaults to 0.0.
        workload_capacity (int, optional): Set maximum number of adressable clients per nurse. Defaults to 7.
        p_training_skills (int, optional): Set the chosen policy of training skills. Defaults to -1.

    Returns:
        dict: dict of results as given by the model's print function with data used addition
    """
    nb_needs = len(df_og_need)
    nb_inter = len(list_available_inter)
    data = {}

    # Initialization
    data["og_time_matrix_car"] = [
        [4000 for i in range(nb_needs + nb_inter)] for k in range(nb_needs + nb_inter)
    ]
    data["og_time_matrix_bike"] = [
        [4000 for i in range(nb_needs + nb_inter)] for k in range(nb_needs + nb_inter)
    ]

    # Create original time matrix
    for i in range(nb_needs):
        need_id_client = df_og_need["ID Client"][i]
        need_idx_client = client_id_to_idx[need_id_client]

        for j in range(nb_needs):
            other_id_client = df_og_need["ID Client"][j]
            other_idx_client = client_id_to_idx[other_id_client]

            travel_time_car = full_time_matrix_car[need_idx_client, other_idx_client]
            travel_time_bike = full_time_matrix_bike[need_idx_client, other_idx_client]
            data["og_time_matrix_car"][i][j] = travel_time_car
            data["og_time_matrix_bike"][i][j] = travel_time_bike

        for j in range(nb_inter):
            id_inter = list_available_inter[j]
            idx_inter = inter_id_to_idx[id_inter]

            travel_time_car = full_time_matrix_car[need_idx_client][
                nb_full_client + idx_inter
            ]
            travel_time_bike = full_time_matrix_bike[need_idx_client][
                nb_full_client + idx_inter
            ]

            data["og_time_matrix_car"][i][nb_needs + j] = travel_time_car
            data["og_time_matrix_car"][nb_needs + j][i] = data["og_time_matrix_car"][i][
                nb_needs + j
            ]
            data["og_time_matrix_bike"][i][nb_needs + j] = travel_time_bike
            data["og_time_matrix_bike"][nb_needs + j][i] = data["og_time_matrix_bike"][
                i
            ][nb_needs + j]
    for i in range(nb_needs + nb_inter):
        data["og_time_matrix_car"][i][i] = 0
        data["og_time_matrix_bike"][i][i] = 0

    # Create new time matrix from orginial adding synthetic travel times
    (
        data["time_matrix_car"],
        data["time_matrix_bike"],
        synth_client_needs_df,
        synth_available_inter_df,
    ) = add_actors(
        df_og_need,
        df_inter,
        list_available_inter,
        data["og_time_matrix_car"],
        data["og_time_matrix_bike"],
        nb_synth_clients,
        nb_synth_inter,
        client_locations,
        inter_locations,
        inter_synth_constraints=inter_constraints,
        date=date,
    )

    if inter_constraints:
        nb_synth_inter = len(synth_available_inter_df)

    data["service_times"] = [
        0 for i in range(nb_needs + nb_inter + nb_synth_clients + nb_synth_inter)
    ]
    data["time_windows"] = [
        (0, 1440)
        for i in range(nb_needs + nb_inter + nb_synth_clients + nb_synth_inter)
    ]

    data["num_vehicles"] = nb_inter + nb_synth_inter

    if inter_constraints:
        list_is_car = get_is_car(list_available_inter, df_inter)
        if nb_synth_inter > 0:
            list_is_car = [*list_is_car, *get_synth_is_car(synth_available_inter_df)]
        data["is_car"] = list_is_car
    else:
        data["is_car"] = [1 for i in range(nb_inter + nb_synth_inter)]
    data["starts"] = [
        nb_needs + nb_synth_clients + i for i in range(nb_inter + nb_synth_inter)
    ]
    data["ends"] = [
        nb_needs + nb_synth_clients + i for i in range(nb_inter + nb_synth_inter)
    ]

    if inter_constraints:
        list_inter_skills = get_inter_skills(list_available_inter, df_inter)
        if nb_synth_inter > 0:
            list_inter_skills = [
                *list_inter_skills,
                *get_synth_inter_skills(synth_available_inter_df),
            ]
        data["vehicle_skills"] = list_inter_skills
    else:
        data["vehicle_skills"] = [list_skills for i in range(nb_inter + nb_synth_inter)]

    if p_training_skills != -1:
        list_inter_skills = get_inter_skills(list_available_inter, df_inter)
        if nb_synth_inter > 0:
            list_inter_skills = [
                *list_inter_skills,
                *get_synth_inter_skills(synth_available_inter_df),
            ]

        if p_training_skills == 2:
            mandatory_skill_1 = ["REPAS"]
            adjusted_inter_skills = add_mandatory_skill(
                mandatory_skill_1, list_inter_skills
            )
        elif p_training_skills == 3:
            mandatory_skill_2 = ["TOILETTE"]
            adjusted_inter_skills = add_mandatory_skill(
                mandatory_skill_2, list_inter_skills
            )
        elif p_training_skills == 4:
            mandatory_skill_1 = ["REPAS"]
            mandatory_skill_2 = ["TOILETTE"]
            adjusted_inter_skills = add_mandatory_skill(
                mandatory_skill_1, list_inter_skills
            )
            adjusted_inter_skills = add_mandatory_skill(
                mandatory_skill_2, adjusted_inter_skills
            )
        elif 0.0 < p_training_skills <= 1.0:
            adjusted_inter_skills = add_proba_skills(
                list_inter_skills, p_training_skills
            )

        data["vehicle_skills"] = adjusted_inter_skills

    data["node_requirements"] = [
        None for i in range(nb_needs + nb_inter + nb_synth_clients + nb_synth_inter)
    ]

    for i in range(nb_needs + nb_synth_clients):
        if i < nb_needs:
            service_time = int(df_og_need["Durée"][i])
            data["service_times"][i] = service_time

            time_window_temp = (
                df_og_need["tranche_horaire"][i]
                .replace("[", "")
                .replace("]", "")
                .replace(",", "")
                .split()
            )
            time_window_start = int(float(time_window_temp[0]) * 60)
            time_window_end = int(float(time_window_temp[1]) * 60)
            data["time_windows"][i] = (time_window_start, time_window_end)

            skills_required = df_og_need["Prestation"][i]
            data["node_requirements"][i] = skills_required
        else:
            service_time = int(synth_client_needs_df["Durée"][i - nb_needs])
            data["service_times"][i] = service_time

            time_window_temp = (
                synth_client_needs_df["tranche_horaire"][i - nb_needs]
                .replace("[", "")
                .replace("]", "")
                .replace(",", "")
                .split()
            )
            time_window_start = int(float(time_window_temp[0]) * 60)
            time_window_end = int(float(time_window_temp[1]) * 60)
            data["time_windows"][i] = (time_window_start, time_window_end)

            skills_required = synth_client_needs_df["Prestation"][i - nb_needs]
            data["node_requirements"][i] = skills_required

    data["time_windows_with_service"] = [
        (
            data["time_windows"][i][0] + data["service_times"][i],
            data["time_windows"][i][1] + data["service_times"][i],
        )
        for i in range(len(data["service_times"]))
    ]

    data["workload_per_node"] = [
        *[1 for i in range(nb_needs + nb_synth_clients)],
        *[0 for k in range(nb_inter + nb_synth_inter)],
    ]  # Useless for now but could be used to differ workload from each node
    data["workload_capacity"] = [
        workload_capacity for i in range(nb_inter + nb_synth_inter)
    ]

    if bike_proportion > 0.00001:
        data["is_car"] = generate_vehicle_type_list(
            nb_synth_inter + nb_inter, bike_proportion
        )

    return data

# 4. Running scenarios


In [157]:
# We study the specific following week
list_dates = [
    "2024-01-15",
    "2024-01-16",
    "2024-01-17",
    "2024-01-18",
    "2024-01-19",
    "2024-01-20",
    "2024-01-21",
]
list_skills = c_l_need["Prestation"].unique()

In [42]:
# Model main function
def main_real_data_with_synth(
    df_need,
    full_time_matrix_car,
    full_time_matrix_bike,
    list_available_inter,
    nb_synth_clients=0,
    nb_synth_inter=0,
    workload_capacity=7,
    bike_proportion=0,
    verbose=0,
    inter_constraints=False,
    date=None,
    p_training_skills=-1,
):
    """Model's main function: 1/ call data creation's function; 2/ set the optimization context and solve; 3/ call output printing function

    Args:
        df_need (pandas.DataFrame): dataframe of original (and not synthetic) clients needs for a given date
        full_time_matrix_car (2D array): original mapping of travel time by car between all clients and inter
        full_time_matrix_bike (2D array): original mapping of travel time by bike between all clients and inter
        list_available_inter (list(int)): list of original (and not synthetic) inter available for the given date
        nb_synth_clients (int, optional): Number of synthetic clients to add. Defaults to 0.
        nb_synth_inter (int, optional): Number of synthetic inter to add. Defaults to 0.
        workload_capacity (int, optional): Set maximum number of adressable clients per nurse. Defaults to 7.
        bike_proportion (int, optional): Set proportion of available bikes. Defaults to 0.0.
        verbose (int, optional): 0: prints overall results for the day; 1: print detailed routes. Defaults to 0.
        inter_constraints (bool, optional): Search for each inter constraints (skills, vehicle types and availability) or not (default). Defaults to False.
        date (_type_, optional): Date of the optimization, format: "YYYY-MM-DD". Defaults to None.
        p_training_skills (int, optional): Set the chosen policy of training skills. Defaults to -1.

    Returns:
        _type_: _description_
    """
    # Instantiate the data problem.
    data = create_real_data_model_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        list_available_inter,
        nb_synth_clients=nb_synth_clients,
        nb_synth_inter=nb_synth_inter,
        bike_proportion=bike_proportion,
        workload_capacity=workload_capacity
        + 1,  # To count the initial double travel (from home / to home)
        inter_constraints=inter_constraints,
        date=date,
        p_training_skills=p_training_skills,
    )

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["time_matrix_car"]), data["num_vehicles"], data["starts"], data["ends"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def time_callback(from_index, to_index, type):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        service_time = data["service_times"][to_node]
        if type == 1:  # Car
            return data["time_matrix_car"][from_node][to_node] + service_time
            # return data["time_matrix_car"][from_node][to_node]
        else:
            return data["time_matrix_bike"][from_node][to_node] + service_time
            # return data["time_matrix_bike"][from_node][to_node]

    car_callback = partial(time_callback, type=1)
    bike_callback = partial(time_callback, type=0)

    car_index = routing.RegisterTransitCallback(car_callback)
    bike_index = routing.RegisterTransitCallback(bike_callback)

    list_index = []
    for k in range(data["num_vehicles"]):
        if data["is_car"][k] == 1:
            routing.SetArcCostEvaluatorOfVehicle(car_index, k)
            list_index.append(car_index)
        else:
            routing.SetArcCostEvaluatorOfVehicle(bike_index, k)
            list_index.append(bike_index)

    # Enforce node requirements based on vehicle skills
    for node_idx, requirement in enumerate(data["node_requirements"]):
        if requirement:  # Check if the node has specific skill requirements
            allowed_vehicles = [
                vehicle_id
                for vehicle_id, skills in enumerate(data["vehicle_skills"])
                if requirement in skills
            ]
            manager_idx = manager.NodeToIndex(node_idx)
            if manager_idx >= 0:  # Ensure manager index is valid
                routing.SetAllowedVehiclesForIndex(allowed_vehicles, manager_idx)

    # Add Time constraint.
    dimension_name = "Time"
    routing.AddDimensionWithVehicleTransitAndCapacity(
        list_index,
        10000,  # no slack
        [10000 for i in range(data["num_vehicles"])],  # vehicle maximum travel time
        True,  # start cumul to zero
        dimension_name,
    )

    time_dimension = routing.GetDimensionOrDie(dimension_name)

    def workload_callback(from_index):
        return 1

    workload_index = routing.RegisterUnaryTransitCallback(workload_callback)
    routing.AddDimensionWithVehicleCapacity(
        workload_index,
        slack_max=0,  # No slack
        vehicle_capacities=data["workload_capacity"],  # Maximum "load" per vehicle
        fix_start_cumul_to_zero=True,
        name="LoadBalance",
    )

    # Allow to drop nodes.
    penalty = 10000
    nb_needs = len(df_need) + nb_synth_clients
    for node in range(nb_needs):
        routing.AddDisjunction([manager.NodeToIndex(node)], penalty)

    # Add time window constraints for each location except start and end.
    for location_idx, time_window in enumerate(data["time_windows_with_service"]):
        if location_idx in data["starts"]:
            continue
        if location_idx in data["ends"]:
            continue
        index = manager.NodeToIndex(location_idx)
        start_time_window = time_window[0]
        end_time_window = time_window[1]
        time_dimension.CumulVar(index).SetRange(start_time_window, end_time_window)

    # Instantiate route start and end times to produce feasible times.
    for i in range(data["num_vehicles"]):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i))
        )
        routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    # search_parameters.log_search = True
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        # print_solution(data, manager, routing, solution)
        dict_result = print_detailed_solution_real_data(
            data, manager, routing, solution, list_available_inter, verbose=verbose
        )
        dict_result["data"] = data
        return dict_result

## 4.1.A. Pas de contraintes


In [175]:
list_scenario_7_1_a = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")

    dict_results = main_real_data_with_synth(
        df_need, full_time_matrix_car, full_time_matrix_bike, full_inter_list, verbose=1
    )

    print("\n\n\n")

    list_scenario_7_1_a.append(dict_results)

Date: 2024-01-15
Dropped nodes: (Total: 0)
Route for vehicle 1452747150 (Type: Car; Skills: ['REPAS' 'TOILETTE' 'VIE SOCIALE' 'AIDE MENAGERE'
 'ACCOMPAGNEMENTS COURSES' 'FEMME DE MENAGE' "GARDE D'ENFANTS"]):
* Location 103 (Arrival: 0h00, End of Service Time (Service time: 0) (Skills required: None): 0h00, Break: 0h00, Departure: 0h00, Next travel time: 0) -> 
* Location 104 (Arrival: 0h00)
Number of small breaks (<30min): 0; Number of big breaks (>30min): 0
Total commuting time for vehicle 1452747150: 0 minutes


Route for vehicle 480302361 (Type: Car; Skills: ['REPAS' 'TOILETTE' 'VIE SOCIALE' 'AIDE MENAGERE'
 'ACCOMPAGNEMENTS COURSES' 'FEMME DE MENAGE' "GARDE D'ENFANTS"]):
* Location 104 (Arrival: 0h00, End of Service Time (Service time: 0) (Skills required: None): 0h00, Break: 6h58, Departure: 6h58, Next travel time: 2) -> 
* Location 13 (Arrival: 7h00, End of Service Time (Service time: 30) (Skills required: TOILETTE): 7h30, Break: 0h00, Departure: 7h30, Next travel time: 1) -> 
* 

In [104]:
total_nb_com = 0
total_com_tim = 0
total_visits = 0
total_drop = 0
avg_l_day = 0
tot_breaks = 0
tot_co2 = 0

list_scenario = list_scenario_7_1_a
for k in range(len(list_scenario)):
    total_com_tim += list_scenario[k]["total_commuting_time"]
    total_visits += list_scenario[k]["nb_visits"]
    total_nb_com += list_scenario[k]["nb_visits"] + list_scenario[k]["nb_inter_used"]
    total_drop += list_scenario[k]["nb_dropped_nodes"]
    avg_l_day += list_scenario[k]["avg_length_day"]
    tot_breaks += list_scenario[k]["nb_break_sup"]
    tot_co2 += list_scenario[k]["eq_co2_cars"] + list_scenario[k]["eq_co2_bikes"]

print(total_com_tim)
print(avg_l_day / len(list_scenario_7_1_a))
print(total_com_tim / total_nb_com)
print(tot_breaks)
print(tot_co2)
print(total_visits)

2237
600.3877551020407
2.9241830065359475
179
212.51499999999996
641


## 4.1.B. Avec contraintes


In [44]:
list_scenario_7_1_b = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")
    list_available_inter = get_list_og_available(date, df_inter)

    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        list_available_inter,
        inter_constraints=True,
        workload_capacity=7,
        verbose=0,
        date=date,
    )

    print("\n\n\n")

    list_scenario_7_1_b.append(dict_results)

Date: 2024-01-15


Dropped nodes: 21; 45; 62; (Total: 3)
Total available inter: 17
Total inter who worked: 17
Toal number of visited nodes: 100
Overall total commuting time: 658 minutes
Number of cars used: 14
Overall total commuting time for cars: 480 minutes (45.6 kgCO2)
Number of bikes used: 3
Overall total commuting time for bikes: 178 minutes (0.11866666666666666 kgCO2)
Total number of small breaks (<30min): 56; Total number of big breaks (>30min): 27
Average length of a day of work: 10h07




Date: 2024-01-16
Dropped nodes: 17; 88; (Total: 2)
Total available inter: 18
Total inter who worked: 18
Toal number of visited nodes: 109
Overall total commuting time: 706 minutes
Number of cars used: 14
Overall total commuting time for cars: 610 minutes (57.95 kgCO2)
Number of bikes used: 4
Overall total commuting time for bikes: 96 minutes (0.06399999999999999 kgCO2)
Total number of small breaks (<30min): 70; Total number of big breaks (>30min): 21
Average length of a day of work: 10h30




Date: 2024-01-17


In [100]:
total_nb_com = 0
total_com_tim = 0
total_visits = 0
total_drop = 0
avg_l_day = 0
tot_breaks = 0
tot_co2 = 0

list_scenario = list_scenario_7_1_b
for k in range(len(list_scenario)):
    total_com_tim += list_scenario[k]["total_commuting_time"]
    total_visits += list_scenario[k]["nb_visits"]
    total_nb_com += list_scenario[k]["nb_visits"] + list_scenario[k]["nb_inter_used"]
    total_drop += list_scenario[k]["nb_dropped_nodes"]
    avg_l_day += list_scenario[k]["avg_length_day"]
    tot_breaks += list_scenario[k]["nb_break_sup"]
    tot_co2 += list_scenario[k]["eq_co2_cars"] + list_scenario[k]["eq_co2_bikes"]

print(total_com_tim)
print(avg_l_day / len(list_scenario_7_1_b))
print(total_com_tim / total_nb_com)
print(tot_breaks)
print(tot_co2)
print(total_visits)

3738
640.8018440709618
5.25
134
300.774
614


## 4.2. Proportions de vélo disponibles (avec ou sans contraintes additionnelles)


### 4.2.A. Proportion: 0%


In [158]:
list_scenario_7_2_a = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")

    # Reminder: inter_constraints = False sets the reference to 'no constraints' and bike_proportion builds on top
    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        full_inter_list,
        inter_constraints=False,
        bike_proportion=0.0,
        verbose=0,
    )

    print("\n\n\n")

    list_scenario_7_2_a.append(dict_results)

Date: 2024-01-15
Dropped nodes: (Total: 0)
Total available inter: 24
Total inter who worked: 18
Toal number of visited nodes: 103
Overall total commuting time: 297 minutes
Number of cars used: 18
Overall total commuting time for cars: 297 minutes (28.214999999999996 kgCO2)
Number of bikes used: 0
Overall total commuting time for bikes: 0 minutes (0.0 kgCO2)
Total number of small breaks (<30min): 63; Total number of big breaks (>30min): 22
Average length of a day of work: 10h26




Date: 2024-01-16
Dropped nodes: (Total: 0)
Total available inter: 24
Total inter who worked: 20
Toal number of visited nodes: 111
Overall total commuting time: 385 minutes
Number of cars used: 20
Overall total commuting time for cars: 385 minutes (36.575 kgCO2)
Number of bikes used: 0
Overall total commuting time for bikes: 0 minutes (0.0 kgCO2)
Total number of small breaks (<30min): 60; Total number of big breaks (>30min): 31
Average length of a day of work: 10h51




Date: 2024-01-17
Dropped nodes: (Total: 

In [163]:
total_nb_com = 0
total_com_tim = 0
total_visits = 0
total_drop = 0
avg_l_day = 0
tot_breaks = 0
tot_co2 = 0
nb_cars = 0
nb_bikes = 0

list_scenario = list_scenario_7_2_a
for k in range(len(list_scenario)):
    total_com_tim += list_scenario[k]["total_commuting_time"]
    total_visits += list_scenario[k]["nb_visits"]
    total_nb_com += list_scenario[k]["nb_visits"] + list_scenario[k]["nb_inter_used"]
    total_drop += list_scenario[k]["nb_dropped_nodes"]
    avg_l_day += list_scenario[k]["avg_length_day"]
    tot_breaks += list_scenario[k]["nb_break_sup"]
    tot_co2 += list_scenario[k]["eq_co2_cars"] + list_scenario[k]["eq_co2_bikes"]
    nb_cars += list_scenario[k]["nb_cars_used"]
    nb_bikes += list_scenario[k]["nb_bikes_used"]

print(total_com_tim)
print(avg_l_day / len(list_scenario_7_1_b))
print(total_com_tim / total_nb_com)
print(tot_breaks)
print(tot_co2)
print(total_visits)
print(nb_cars)
print(nb_bikes)

2036
623.4045684940644
2.668414154652687
176
193.42
641
122
0


In [46]:
list_scenario_7_2_a_bis = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")
    list_available_inter = get_list_og_available(date, df_inter)

    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        list_available_inter,
        inter_constraints=True,
        workload_capacity=7,
        verbose=0,
        date=date,
        bike_proportion=0.0,
    )

    print("\n\n\n")

    list_scenario_7_2_a_bis.append(dict_results)

Date: 2024-01-15
Dropped nodes: 21; 45; 62; (Total: 3)
Total available inter: 17
Total inter who worked: 17
Toal number of visited nodes: 100
Overall total commuting time: 525 minutes
Number of cars used: 14
Overall total commuting time for cars: 455 minutes (43.225 kgCO2)
Number of bikes used: 3
Overall total commuting time for bikes: 70 minutes (0.04666666666666667 kgCO2)
Total number of small breaks (<30min): 55; Total number of big breaks (>30min): 28
Average length of a day of work: 10h05




Date: 2024-01-16
Dropped nodes: 17; 88; (Total: 2)
Total available inter: 18
Total inter who worked: 18
Toal number of visited nodes: 109
Overall total commuting time: 721 minutes
Number of cars used: 14
Overall total commuting time for cars: 631 minutes (59.94499999999999 kgCO2)
Number of bikes used: 4
Overall total commuting time for bikes: 90 minutes (0.06 kgCO2)
Total number of small breaks (<30min): 73; Total number of big breaks (>30min): 18
Average length of a day of work: 9h54




Dat

### 4.2.B. Proportion: 10%


In [164]:
list_scenario_7_2_b = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")

    # Reminder: inter_constraints = False sets the reference to 'no constraints' and bike_proportion builds on top
    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        full_inter_list,
        inter_constraints=False,
        bike_proportion=0.1,
        verbose=0,
    )

    print("\n\n\n")

    list_scenario_7_2_b.append(dict_results)

Date: 2024-01-15
Dropped nodes: (Total: 0)
Total available inter: 24
Total inter who worked: 20
Toal number of visited nodes: 103
Overall total commuting time: 323 minutes
Number of cars used: 20
Overall total commuting time for cars: 323 minutes (30.684999999999995 kgCO2)
Number of bikes used: 0
Overall total commuting time for bikes: 0 minutes (0.0 kgCO2)
Total number of small breaks (<30min): 54; Total number of big breaks (>30min): 29
Average length of a day of work: 10h49




Date: 2024-01-16
Dropped nodes: (Total: 0)
Total available inter: 24
Total inter who worked: 20
Toal number of visited nodes: 111
Overall total commuting time: 455 minutes
Number of cars used: 19
Overall total commuting time for cars: 438 minutes (41.61 kgCO2)
Number of bikes used: 1
Overall total commuting time for bikes: 17 minutes (0.011333333333333334 kgCO2)
Total number of small breaks (<30min): 59; Total number of big breaks (>30min): 32
Average length of a day of work: 10h46




Date: 2024-01-17
Droppe

In [165]:
total_nb_com = 0
total_com_tim = 0
total_visits = 0
total_drop = 0
avg_l_day = 0
tot_breaks = 0
tot_co2 = 0
nb_cars = 0
nb_bikes = 0

list_scenario = list_scenario_7_2_b
for k in range(len(list_scenario)):
    total_com_tim += list_scenario[k]["total_commuting_time"]
    total_visits += list_scenario[k]["nb_visits"]
    total_nb_com += list_scenario[k]["nb_visits"] + list_scenario[k]["nb_inter_used"]
    total_drop += list_scenario[k]["nb_dropped_nodes"]
    avg_l_day += list_scenario[k]["avg_length_day"]
    tot_breaks += list_scenario[k]["nb_break_sup"]
    tot_co2 += list_scenario[k]["eq_co2_cars"] + list_scenario[k]["eq_co2_bikes"]
    nb_cars += list_scenario[k]["nb_cars_used"]
    nb_bikes += list_scenario[k]["nb_bikes_used"]

print(total_com_tim)
print(avg_l_day / len(list_scenario_7_1_b))
print(total_com_tim / total_nb_com)
print(tot_breaks)
print(tot_co2)
print(total_visits)
print(nb_cars)
print(nb_bikes)

2181
624.9165771571786
2.836150845253576
191
203.516
641
122
6


In [48]:
list_scenario_7_2_b_bis = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")
    list_available_inter = get_list_og_available(date, df_inter)

    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        list_available_inter,
        inter_constraints=True,
        workload_capacity=7,
        verbose=0,
        date=date,
        bike_proportion=0.1,
    )

    print("\n\n\n")

    list_scenario_7_2_b_bis.append(dict_results)

Date: 2024-01-15
Dropped nodes: 67; (Total: 1)
Total available inter: 17
Total inter who worked: 16
Toal number of visited nodes: 102
Overall total commuting time: 556 minutes
Number of cars used: 16
Overall total commuting time for cars: 556 minutes (52.81999999999999 kgCO2)
Number of bikes used: 0
Overall total commuting time for bikes: 0 minutes (0.0 kgCO2)
Total number of small breaks (<30min): 64; Total number of big breaks (>30min): 22
Average length of a day of work: 10h33




Date: 2024-01-16
Dropped nodes: 17; 74; 88; (Total: 3)
Total available inter: 18
Total inter who worked: 16
Toal number of visited nodes: 108
Overall total commuting time: 641 minutes
Number of cars used: 15
Overall total commuting time for cars: 637 minutes (60.515 kgCO2)
Number of bikes used: 1
Overall total commuting time for bikes: 4 minutes (0.0026666666666666666 kgCO2)
Total number of small breaks (<30min): 73; Total number of big breaks (>30min): 19
Average length of a day of work: 10h50




Date: 2

### 4.2.C. Proportion: 25%


In [166]:
list_scenario_7_2_c = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")

    # Reminder: inter_constraints = False sets the reference to 'no constraints' and bike_proportion builds on top
    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        full_inter_list,
        inter_constraints=False,
        bike_proportion=0.25,
        verbose=0,
    )

    print("\n\n\n")

    list_scenario_7_2_c.append(dict_results)

Date: 2024-01-15
Dropped nodes: (Total: 0)
Total available inter: 24
Total inter who worked: 19
Toal number of visited nodes: 103
Overall total commuting time: 532 minutes
Number of cars used: 17
Overall total commuting time for cars: 476 minutes (45.22 kgCO2)
Number of bikes used: 2
Overall total commuting time for bikes: 56 minutes (0.037333333333333336 kgCO2)
Total number of small breaks (<30min): 62; Total number of big breaks (>30min): 22
Average length of a day of work: 9h45




Date: 2024-01-16
Dropped nodes: (Total: 0)
Total available inter: 24
Total inter who worked: 21
Toal number of visited nodes: 111
Overall total commuting time: 418 minutes
Number of cars used: 18
Overall total commuting time for cars: 403 minutes (38.285 kgCO2)
Number of bikes used: 3
Overall total commuting time for bikes: 15 minutes (0.009999999999999998 kgCO2)
Total number of small breaks (<30min): 64; Total number of big breaks (>30min): 26
Average length of a day of work: 10h22




Date: 2024-01-17
D

In [168]:
total_nb_com = 0
total_com_tim = 0
total_visits = 0
total_drop = 0
avg_l_day = 0
tot_breaks = 0
tot_co2 = 0
nb_cars = 0
nb_bikes = 0

list_scenario = list_scenario_7_2_c
for k in range(len(list_scenario)):
    total_com_tim += list_scenario[k]["total_commuting_time"]
    total_visits += list_scenario[k]["nb_visits"]
    total_nb_com += list_scenario[k]["nb_visits"] + list_scenario[k]["nb_inter_used"]
    total_drop += list_scenario[k]["nb_dropped_nodes"]
    avg_l_day += list_scenario[k]["avg_length_day"]
    tot_breaks += list_scenario[k]["nb_break_sup"]
    tot_co2 += list_scenario[k]["eq_co2_cars"] + list_scenario[k]["eq_co2_bikes"]
    nb_cars += list_scenario[k]["nb_cars_used"]
    nb_bikes += list_scenario[k]["nb_bikes_used"]

print(total_com_tim)
print(avg_l_day / len(list_scenario_7_1_b))
print(total_com_tim / total_nb_com)
print(tot_breaks)
print(tot_co2)
print(total_visits)
print(nb_cars)
print(nb_bikes)

2377
607.6377257043168
3.10718954248366
161
207.2313333333333
641
107
17


In [50]:
list_scenario_7_2_c_bis = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")
    list_available_inter = get_list_og_available(date, df_inter)

    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        list_available_inter,
        inter_constraints=True,
        workload_capacity=7,
        verbose=0,
        date=date,
        bike_proportion=0.25,
    )

    print("\n\n\n")

    list_scenario_7_2_c_bis.append(dict_results)

Date: 2024-01-15
Dropped nodes: 21; 45; 62; (Total: 3)
Total available inter: 17
Total inter who worked: 16
Toal number of visited nodes: 100
Overall total commuting time: 587 minutes
Number of cars used: 13
Overall total commuting time for cars: 436 minutes (41.42 kgCO2)
Number of bikes used: 3
Overall total commuting time for bikes: 151 minutes (0.10066666666666665 kgCO2)
Total number of small breaks (<30min): 60; Total number of big breaks (>30min): 24
Average length of a day of work: 11h12




Date: 2024-01-16
Dropped nodes: 17; 74; 88; (Total: 3)
Total available inter: 18
Total inter who worked: 18
Toal number of visited nodes: 108
Overall total commuting time: 645 minutes
Number of cars used: 14
Overall total commuting time for cars: 516 minutes (49.019999999999996 kgCO2)
Number of bikes used: 4
Overall total commuting time for bikes: 129 minutes (0.086 kgCO2)
Total number of small breaks (<30min): 67; Total number of big breaks (>30min): 23
Average length of a day of work: 10h41

### 4.2.D. Proportion: 50%


In [170]:
list_scenario_7_2_d = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")

    # Reminder: inter_constraints = False sets the reference to 'no constraints' and bike_proportion builds on top
    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        full_inter_list,
        inter_constraints=False,
        bike_proportion=0.5,
        verbose=0,
    )

    print("\n\n\n")

    list_scenario_7_2_d.append(dict_results)

Date: 2024-01-15
Dropped nodes: (Total: 0)
Total available inter: 24
Total inter who worked: 19
Toal number of visited nodes: 103
Overall total commuting time: 318 minutes
Number of cars used: 11
Overall total commuting time for cars: 226 minutes (21.47 kgCO2)
Number of bikes used: 8
Overall total commuting time for bikes: 92 minutes (0.06133333333333333 kgCO2)
Total number of small breaks (<30min): 61; Total number of big breaks (>30min): 23
Average length of a day of work: 9h53




Date: 2024-01-16
Dropped nodes: (Total: 0)
Total available inter: 24
Total inter who worked: 19
Toal number of visited nodes: 111
Overall total commuting time: 493 minutes
Number of cars used: 11
Overall total commuting time for cars: 336 minutes (31.919999999999998 kgCO2)
Number of bikes used: 8
Overall total commuting time for bikes: 157 minutes (0.10466666666666666 kgCO2)
Total number of small breaks (<30min): 66; Total number of big breaks (>30min): 26
Average length of a day of work: 10h54




Date: 2

In [171]:
total_nb_com = 0
total_com_tim = 0
total_visits = 0
total_drop = 0
avg_l_day = 0
tot_breaks = 0
tot_co2 = 0
nb_cars = 0
nb_bikes = 0

list_scenario = list_scenario_7_2_d
for k in range(len(list_scenario)):
    total_com_tim += list_scenario[k]["total_commuting_time"]
    total_visits += list_scenario[k]["nb_visits"]
    total_nb_com += list_scenario[k]["nb_visits"] + list_scenario[k]["nb_inter_used"]
    total_drop += list_scenario[k]["nb_dropped_nodes"]
    avg_l_day += list_scenario[k]["avg_length_day"]
    tot_breaks += list_scenario[k]["nb_break_sup"]
    tot_co2 += list_scenario[k]["eq_co2_cars"] + list_scenario[k]["eq_co2_bikes"]
    nb_cars += list_scenario[k]["nb_cars_used"]
    nb_bikes += list_scenario[k]["nb_bikes_used"]

print(total_com_tim)
print(avg_l_day / len(list_scenario_7_1_b))
print(total_com_tim / total_nb_com)
print(tot_breaks)
print(tot_co2)
print(total_visits)
print(nb_cars)
print(nb_bikes)

2504
620.2794372809852
3.3077939233817704
159
189.58133333333333
641
73
43


In [52]:
list_scenario_7_2_d_bis = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")
    list_available_inter = get_list_og_available(date, df_inter)

    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        list_available_inter,
        inter_constraints=True,
        workload_capacity=7,
        verbose=0,
        date=date,
        bike_proportion=0.5,
    )

    print("\n\n\n")

    list_scenario_7_2_d_bis.append(dict_results)

Date: 2024-01-15
Dropped nodes: (Total: 0)
Total available inter: 17
Total inter who worked: 16
Toal number of visited nodes: 103
Overall total commuting time: 772 minutes
Number of cars used: 9
Overall total commuting time for cars: 273 minutes (25.934999999999995 kgCO2)
Number of bikes used: 7
Overall total commuting time for bikes: 499 minutes (0.33266666666666667 kgCO2)
Total number of small breaks (<30min): 70; Total number of big breaks (>30min): 17
Average length of a day of work: 10h18




Date: 2024-01-16
Dropped nodes: 17; 19; 74; 88; (Total: 4)
Total available inter: 18
Total inter who worked: 16
Toal number of visited nodes: 107
Overall total commuting time: 1021 minutes
Number of cars used: 9
Overall total commuting time for cars: 473 minutes (44.934999999999995 kgCO2)
Number of bikes used: 7
Overall total commuting time for bikes: 548 minutes (0.36533333333333334 kgCO2)
Total number of small breaks (<30min): 73; Total number of big breaks (>30min): 18
Average length of a 

## 4.3. Politiques de Training pour les employés


In [179]:
list_scenario_7_3_0 = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")

    # Reminder: inter_constraints = False sets the reference to 'no constraints' and bike_proportion builds on top
    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        full_inter_list,
        inter_constraints=False,
        p_training_skills=-1,
        verbose=0,
    )

    print("\n\n\n")

    list_scenario_7_3_0.append(dict_results)

Date: 2024-01-15
Dropped nodes: (Total: 0)
Total available inter: 24
Total inter who worked: 18
Toal number of visited nodes: 103
Overall total commuting time: 622 minutes
Number of cars used: 18
Overall total commuting time for cars: 622 minutes (59.089999999999996 kgCO2)
Number of bikes used: 0
Overall total commuting time for bikes: 0 minutes (0.0 kgCO2)
Total number of small breaks (<30min): 68; Total number of big breaks (>30min): 17
Average length of a day of work: 9h39




Date: 2024-01-16
Dropped nodes: (Total: 0)
Total available inter: 24
Total inter who worked: 20
Toal number of visited nodes: 111
Overall total commuting time: 384 minutes
Number of cars used: 20
Overall total commuting time for cars: 384 minutes (36.48 kgCO2)
Number of bikes used: 0
Overall total commuting time for bikes: 0 minutes (0.0 kgCO2)
Total number of small breaks (<30min): 60; Total number of big breaks (>30min): 31
Average length of a day of work: 10h50




Date: 2024-01-17
Dropped nodes: (Total: 0)

In [180]:
total_nb_com = 0
total_com_tim = 0
total_visits = 0
total_drop = 0
avg_l_day = 0
tot_breaks = 0
tot_co2 = 0
nb_cars = 0
nb_bikes = 0

list_scenario = list_scenario_7_3_0
for k in range(len(list_scenario)):
    total_com_tim += list_scenario[k]["total_commuting_time"]
    total_visits += list_scenario[k]["nb_visits"]
    total_nb_com += list_scenario[k]["nb_visits"] + list_scenario[k]["nb_inter_used"]
    total_drop += list_scenario[k]["nb_dropped_nodes"]
    avg_l_day += list_scenario[k]["avg_length_day"]
    tot_breaks += list_scenario[k]["nb_break_sup"]
    tot_co2 += list_scenario[k]["eq_co2_cars"] + list_scenario[k]["eq_co2_bikes"]
    nb_cars += list_scenario[k]["nb_cars_used"]
    nb_bikes += list_scenario[k]["nb_bikes_used"]

print(total_com_tim)
print(avg_l_day / len(list_scenario_7_1_b))
print(total_com_tim / total_nb_com)
print(tot_breaks)
print(tot_co2)
print(total_visits)
print(nb_cars)
print(nb_bikes)

2337
619.7625050020008
3.0629095674967233
174
222.015
641
122
0


### 4.3.A. Ajout REPAS


In [177]:
list_scenario_7_3_a = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")

    # Reminder: inter_constraints = False sets the reference to 'no constraints' and bike_proportion builds on top
    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        full_inter_list,
        inter_constraints=False,
        p_training_skills=2,
        verbose=0,
    )

    print("\n\n\n")

    list_scenario_7_3_a.append(dict_results)

Date: 2024-01-15
Dropped nodes: (Total: 0)
Total available inter: 24
Total inter who worked: 17
Toal number of visited nodes: 103
Overall total commuting time: 356 minutes
Number of cars used: 17
Overall total commuting time for cars: 356 minutes (33.82 kgCO2)
Number of bikes used: 0
Overall total commuting time for bikes: 0 minutes (0.0 kgCO2)
Total number of small breaks (<30min): 65; Total number of big breaks (>30min): 21
Average length of a day of work: 10h40




Date: 2024-01-16
Dropped nodes: (Total: 0)
Total available inter: 24
Total inter who worked: 19
Toal number of visited nodes: 111
Overall total commuting time: 458 minutes
Number of cars used: 19
Overall total commuting time for cars: 458 minutes (43.51 kgCO2)
Number of bikes used: 0
Overall total commuting time for bikes: 0 minutes (0.0 kgCO2)
Total number of small breaks (<30min): 69; Total number of big breaks (>30min): 23
Average length of a day of work: 10h26




Date: 2024-01-17
Dropped nodes: (Total: 0)
Total avail

In [178]:
total_nb_com = 0
total_com_tim = 0
total_visits = 0
total_drop = 0
avg_l_day = 0
tot_breaks = 0
tot_co2 = 0
nb_cars = 0
nb_bikes = 0

list_scenario = list_scenario_7_3_a
for k in range(len(list_scenario)):
    total_com_tim += list_scenario[k]["total_commuting_time"]
    total_visits += list_scenario[k]["nb_visits"]
    total_nb_com += list_scenario[k]["nb_visits"] + list_scenario[k]["nb_inter_used"]
    total_drop += list_scenario[k]["nb_dropped_nodes"]
    avg_l_day += list_scenario[k]["avg_length_day"]
    tot_breaks += list_scenario[k]["nb_break_sup"]
    tot_co2 += list_scenario[k]["eq_co2_cars"] + list_scenario[k]["eq_co2_bikes"]
    nb_cars += list_scenario[k]["nb_cars_used"]
    nb_bikes += list_scenario[k]["nb_bikes_used"]

print(total_com_tim)
print(avg_l_day / len(list_scenario_7_1_b))
print(total_com_tim / total_nb_com)
print(tot_breaks)
print(tot_co2)
print(total_visits)
print(nb_cars)
print(nb_bikes)

2544
619.8255320144252
3.3606340819022456
150
241.67999999999995
641
116
0


In [54]:
list_scenario_7_3_a_bis = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")
    list_available_inter = get_list_og_available(date, df_inter)

    # Reminder: inter_constraints = False sets the reference to 'no constraints' and bike_proportion builds on top
    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        list_available_inter,
        inter_constraints=True,
        date=date,
        p_training_skills=2,
        verbose=0,
    )

    print("\n\n\n")

    list_scenario_7_3_a_bis.append(dict_results)

Date: 2024-01-15
Dropped nodes: 67; (Total: 1)
Total available inter: 17
Total inter who worked: 16
Toal number of visited nodes: 102
Overall total commuting time: 636 minutes
Number of cars used: 13
Overall total commuting time for cars: 477 minutes (45.315 kgCO2)
Number of bikes used: 3
Overall total commuting time for bikes: 159 minutes (0.10600000000000001 kgCO2)
Total number of small breaks (<30min): 60; Total number of big breaks (>30min): 26
Average length of a day of work: 10h50




Date: 2024-01-16
Dropped nodes: 17; 88; (Total: 2)
Total available inter: 18
Total inter who worked: 18
Toal number of visited nodes: 109
Overall total commuting time: 706 minutes
Number of cars used: 14
Overall total commuting time for cars: 610 minutes (57.95 kgCO2)
Number of bikes used: 4
Overall total commuting time for bikes: 96 minutes (0.06399999999999999 kgCO2)
Total number of small breaks (<30min): 70; Total number of big breaks (>30min): 21
Average length of a day of work: 10h30




Date: 

### 4.3.B. Ajout TOILETTE


In [181]:
list_scenario_7_3_b = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")

    # Reminder: inter_constraints = False sets the reference to 'no constraints' and bike_proportion builds on top
    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        full_inter_list,
        inter_constraints=False,
        p_training_skills=3,
        verbose=0,
    )

    print("\n\n\n")

    list_scenario_7_3_b.append(dict_results)

Date: 2024-01-15


Dropped nodes: (Total: 0)
Total available inter: 24
Total inter who worked: 17
Toal number of visited nodes: 103
Overall total commuting time: 363 minutes
Number of cars used: 17
Overall total commuting time for cars: 363 minutes (34.485 kgCO2)
Number of bikes used: 0
Overall total commuting time for bikes: 0 minutes (0.0 kgCO2)
Total number of small breaks (<30min): 66; Total number of big breaks (>30min): 20
Average length of a day of work: 10h33




Date: 2024-01-16
Dropped nodes: (Total: 0)
Total available inter: 24
Total inter who worked: 19
Toal number of visited nodes: 111
Overall total commuting time: 458 minutes
Number of cars used: 19
Overall total commuting time for cars: 458 minutes (43.51 kgCO2)
Number of bikes used: 0
Overall total commuting time for bikes: 0 minutes (0.0 kgCO2)
Total number of small breaks (<30min): 69; Total number of big breaks (>30min): 23
Average length of a day of work: 10h26




Date: 2024-01-17
Dropped nodes: (Total: 0)
Total available inter: 24
T

In [182]:
total_nb_com = 0
total_com_tim = 0
total_visits = 0
total_drop = 0
avg_l_day = 0
tot_breaks = 0
tot_co2 = 0
nb_cars = 0
nb_bikes = 0

list_scenario = list_scenario_7_3_b
for k in range(len(list_scenario)):
    total_com_tim += list_scenario[k]["total_commuting_time"]
    total_visits += list_scenario[k]["nb_visits"]
    total_nb_com += list_scenario[k]["nb_visits"] + list_scenario[k]["nb_inter_used"]
    total_drop += list_scenario[k]["nb_dropped_nodes"]
    avg_l_day += list_scenario[k]["avg_length_day"]
    tot_breaks += list_scenario[k]["nb_break_sup"]
    tot_co2 += list_scenario[k]["eq_co2_cars"] + list_scenario[k]["eq_co2_bikes"]
    nb_cars += list_scenario[k]["nb_cars_used"]
    nb_bikes += list_scenario[k]["nb_bikes_used"]

print(total_com_tim)
print(avg_l_day / len(list_scenario_7_1_b))
print(total_com_tim / total_nb_com)
print(tot_breaks)
print(tot_co2)
print(total_visits)
print(nb_cars)
print(nb_bikes)

2377
623.6860886054827
3.148344370860927
149
225.815
641
114
0


In [56]:
list_scenario_7_3_b_bis = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")
    list_available_inter = get_list_og_available(date, df_inter)

    # Reminder: inter_constraints = False sets the reference to 'no constraints' and bike_proportion builds on top
    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        list_available_inter,
        inter_constraints=True,
        date=date,
        p_training_skills=3,
        verbose=0,
    )

    print("\n\n\n")

    list_scenario_7_3_b_bis.append(dict_results)

Date: 2024-01-15
Dropped nodes: 21; 45; 62; (Total: 3)
Total available inter: 17
Total inter who worked: 17
Toal number of visited nodes: 100
Overall total commuting time: 525 minutes
Number of cars used: 14
Overall total commuting time for cars: 455 minutes (43.225 kgCO2)
Number of bikes used: 3
Overall total commuting time for bikes: 70 minutes (0.04666666666666667 kgCO2)
Total number of small breaks (<30min): 55; Total number of big breaks (>30min): 28
Average length of a day of work: 10h05




Date: 2024-01-16
Dropped nodes: 17; 88; (Total: 2)
Total available inter: 18
Total inter who worked: 18
Toal number of visited nodes: 109
Overall total commuting time: 721 minutes
Number of cars used: 14
Overall total commuting time for cars: 631 minutes (59.94499999999999 kgCO2)
Number of bikes used: 4
Overall total commuting time for bikes: 90 minutes (0.06 kgCO2)
Total number of small breaks (<30min): 73; Total number of big breaks (>30min): 18
Average length of a day of work: 9h54




Dat

### 4.3.C. Ajout REPAS et TOILETTE


In [183]:
list_scenario_7_3_c = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")

    # Reminder: inter_constraints = False sets the reference to 'no constraints' and bike_proportion builds on top
    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        full_inter_list,
        inter_constraints=False,
        p_training_skills=4,
        verbose=0,
    )

    print("\n\n\n")

    list_scenario_7_3_c.append(dict_results)

Date: 2024-01-15
Dropped nodes: (Total: 0)
Total available inter: 24
Total inter who worked: 17
Toal number of visited nodes: 103
Overall total commuting time: 356 minutes
Number of cars used: 17
Overall total commuting time for cars: 356 minutes (33.82 kgCO2)
Number of bikes used: 0
Overall total commuting time for bikes: 0 minutes (0.0 kgCO2)
Total number of small breaks (<30min): 65; Total number of big breaks (>30min): 21
Average length of a day of work: 10h40




Date: 2024-01-16
Dropped nodes: (Total: 0)
Total available inter: 24
Total inter who worked: 19
Toal number of visited nodes: 111
Overall total commuting time: 458 minutes
Number of cars used: 19
Overall total commuting time for cars: 458 minutes (43.51 kgCO2)
Number of bikes used: 0
Overall total commuting time for bikes: 0 minutes (0.0 kgCO2)
Total number of small breaks (<30min): 69; Total number of big breaks (>30min): 23
Average length of a day of work: 10h26




Date: 2024-01-17
Dropped nodes: (Total: 0)
Total avail

In [184]:
total_nb_com = 0
total_com_tim = 0
total_visits = 0
total_drop = 0
avg_l_day = 0
tot_breaks = 0
tot_co2 = 0
nb_cars = 0
nb_bikes = 0

list_scenario = list_scenario_7_3_c
for k in range(len(list_scenario)):
    total_com_tim += list_scenario[k]["total_commuting_time"]
    total_visits += list_scenario[k]["nb_visits"]
    total_nb_com += list_scenario[k]["nb_visits"] + list_scenario[k]["nb_inter_used"]
    total_drop += list_scenario[k]["nb_dropped_nodes"]
    avg_l_day += list_scenario[k]["avg_length_day"]
    tot_breaks += list_scenario[k]["nb_break_sup"]
    tot_co2 += list_scenario[k]["eq_co2_cars"] + list_scenario[k]["eq_co2_bikes"]
    nb_cars += list_scenario[k]["nb_cars_used"]
    nb_bikes += list_scenario[k]["nb_bikes_used"]

print(total_com_tim)
print(avg_l_day / len(list_scenario_7_1_b))
print(total_com_tim / total_nb_com)
print(tot_breaks)
print(tot_co2)
print(total_visits)
print(nb_cars)
print(nb_bikes)

2547
608.5555563925975
3.34251968503937
156
241.965
641
121
0


In [58]:
list_scenario_7_3_c_bis = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")
    list_available_inter = get_list_og_available(date, df_inter)

    # Reminder: inter_constraints = False sets the reference to 'no constraints' and bike_proportion builds on top
    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        list_available_inter,
        inter_constraints=True,
        date=date,
        p_training_skills=4,
        verbose=0,
    )

    print("\n\n\n")

    list_scenario_7_3_c_bis.append(dict_results)

Date: 2024-01-15
Dropped nodes: 67; (Total: 1)
Total available inter: 17
Total inter who worked: 16
Toal number of visited nodes: 102
Overall total commuting time: 660 minutes
Number of cars used: 13
Overall total commuting time for cars: 500 minutes (47.5 kgCO2)
Number of bikes used: 3
Overall total commuting time for bikes: 160 minutes (0.10666666666666666 kgCO2)
Total number of small breaks (<30min): 59; Total number of big breaks (>30min): 27
Average length of a day of work: 10h57




Date: 2024-01-16
Dropped nodes: 17; 88; (Total: 2)
Total available inter: 18
Total inter who worked: 18
Toal number of visited nodes: 109
Overall total commuting time: 694 minutes
Number of cars used: 14
Overall total commuting time for cars: 613 minutes (58.235 kgCO2)
Number of bikes used: 4
Overall total commuting time for bikes: 81 minutes (0.054000000000000006 kgCO2)
Total number of small breaks (<30min): 71; Total number of big breaks (>30min): 20
Average length of a day of work: 10h32




Date: 

### 4.3.D. Training 20%


In [185]:
list_scenario_7_3_d = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")

    # Reminder: inter_constraints = False sets the reference to 'no constraints' and bike_proportion builds on top
    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        full_inter_list,
        inter_constraints=False,
        p_training_skills=0.2,
        verbose=0,
    )

    print("\n\n\n")

    list_scenario_7_3_d.append(dict_results)

Date: 2024-01-15
Dropped nodes: (Total: 0)
Total available inter: 24
Total inter who worked: 17
Toal number of visited nodes: 103
Overall total commuting time: 368 minutes
Number of cars used: 17
Overall total commuting time for cars: 368 minutes (34.959999999999994 kgCO2)
Number of bikes used: 0
Overall total commuting time for bikes: 0 minutes (0.0 kgCO2)
Total number of small breaks (<30min): 68; Total number of big breaks (>30min): 18
Average length of a day of work: 10h27




Date: 2024-01-16
Dropped nodes: (Total: 0)
Total available inter: 24
Total inter who worked: 18
Toal number of visited nodes: 111
Overall total commuting time: 618 minutes
Number of cars used: 18
Overall total commuting time for cars: 618 minutes (58.71 kgCO2)
Number of bikes used: 0
Overall total commuting time for bikes: 0 minutes (0.0 kgCO2)
Total number of small breaks (<30min): 73; Total number of big breaks (>30min): 20
Average length of a day of work: 10h14




Date: 2024-01-17
Dropped nodes: (Total: 0

In [186]:
total_nb_com = 0
total_com_tim = 0
total_visits = 0
total_drop = 0
avg_l_day = 0
tot_breaks = 0
tot_co2 = 0
nb_cars = 0
nb_bikes = 0

list_scenario = list_scenario_7_3_d
for k in range(len(list_scenario)):
    total_com_tim += list_scenario[k]["total_commuting_time"]
    total_visits += list_scenario[k]["nb_visits"]
    total_nb_com += list_scenario[k]["nb_visits"] + list_scenario[k]["nb_inter_used"]
    total_drop += list_scenario[k]["nb_dropped_nodes"]
    avg_l_day += list_scenario[k]["avg_length_day"]
    tot_breaks += list_scenario[k]["nb_break_sup"]
    tot_co2 += list_scenario[k]["eq_co2_cars"] + list_scenario[k]["eq_co2_bikes"]
    nb_cars += list_scenario[k]["nb_cars_used"]
    nb_bikes += list_scenario[k]["nb_bikes_used"]

print(total_com_tim)
print(avg_l_day / len(list_scenario_7_1_b))
print(total_com_tim / total_nb_com)
print(tot_breaks)
print(tot_co2)
print(total_visits)
print(nb_cars)
print(nb_bikes)

2462
616.5324550872981
3.2480211081794197
153
233.88999999999996
641
117
0


In [60]:
list_scenario_7_3_d_bis = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")
    list_available_inter = get_list_og_available(date, df_inter)

    # Reminder: inter_constraints = False sets the reference to 'no constraints' and bike_proportion builds on top
    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        list_available_inter,
        inter_constraints=True,
        date=date,
        p_training_skills=0.2,
        verbose=0,
    )

    print("\n\n\n")

    list_scenario_7_3_d_bis.append(dict_results)

Date: 2024-01-15
Dropped nodes: 21; (Total: 1)
Total available inter: 17
Total inter who worked: 17
Toal number of visited nodes: 102
Overall total commuting time: 531 minutes
Number of cars used: 14
Overall total commuting time for cars: 431 minutes (40.945 kgCO2)
Number of bikes used: 3
Overall total commuting time for bikes: 100 minutes (0.06666666666666667 kgCO2)
Total number of small breaks (<30min): 66; Total number of big breaks (>30min): 19
Average length of a day of work: 10h13




Date: 2024-01-16
Dropped nodes: 17; 19; (Total: 2)
Total available inter: 18
Total inter who worked: 17
Toal number of visited nodes: 109
Overall total commuting time: 675 minutes
Number of cars used: 14
Overall total commuting time for cars: 590 minutes (56.05 kgCO2)
Number of bikes used: 3
Overall total commuting time for bikes: 85 minutes (0.056666666666666664 kgCO2)
Total number of small breaks (<30min): 75; Total number of big breaks (>30min): 17
Average length of a day of work: 9h40




Date: 

### 4.3.E. Training 40%


In [187]:
list_scenario_7_3_e = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")

    # Reminder: inter_constraints = False sets the reference to 'no constraints' and bike_proportion builds on top
    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        full_inter_list,
        inter_constraints=False,
        p_training_skills=0.4,
        verbose=0,
    )

    print("\n\n\n")

    list_scenario_7_3_e.append(dict_results)

Date: 2024-01-15
Dropped nodes: (Total: 0)
Total available inter: 24
Total inter who worked: 18
Toal number of visited nodes: 103
Overall total commuting time: 357 minutes
Number of cars used: 18
Overall total commuting time for cars: 357 minutes (33.915 kgCO2)
Number of bikes used: 0
Overall total commuting time for bikes: 0 minutes (0.0 kgCO2)
Total number of small breaks (<30min): 59; Total number of big breaks (>30min): 26
Average length of a day of work: 10h37




Date: 2024-01-16
Dropped nodes: (Total: 0)
Total available inter: 24
Total inter who worked: 21
Toal number of visited nodes: 111
Overall total commuting time: 433 minutes
Number of cars used: 21
Overall total commuting time for cars: 433 minutes (41.135 kgCO2)
Number of bikes used: 0
Overall total commuting time for bikes: 0 minutes (0.0 kgCO2)
Total number of small breaks (<30min): 59; Total number of big breaks (>30min): 31
Average length of a day of work: 10h02




Date: 2024-01-17
Dropped nodes: (Total: 0)
Total ava

In [188]:
total_nb_com = 0
total_com_tim = 0
total_visits = 0
total_drop = 0
avg_l_day = 0
tot_breaks = 0
tot_co2 = 0
nb_cars = 0
nb_bikes = 0

list_scenario = list_scenario_7_3_e
for k in range(len(list_scenario)):
    total_com_tim += list_scenario[k]["total_commuting_time"]
    total_visits += list_scenario[k]["nb_visits"]
    total_nb_com += list_scenario[k]["nb_visits"] + list_scenario[k]["nb_inter_used"]
    total_drop += list_scenario[k]["nb_dropped_nodes"]
    avg_l_day += list_scenario[k]["avg_length_day"]
    tot_breaks += list_scenario[k]["nb_break_sup"]
    tot_co2 += list_scenario[k]["eq_co2_cars"] + list_scenario[k]["eq_co2_bikes"]
    nb_cars += list_scenario[k]["nb_cars_used"]
    nb_bikes += list_scenario[k]["nb_bikes_used"]

print(total_com_tim)
print(avg_l_day / len(list_scenario_7_1_b))
print(total_com_tim / total_nb_com)
print(tot_breaks)
print(tot_co2)
print(total_visits)
print(nb_cars)
print(nb_bikes)

2374
604.7740265336904
3.107329842931937
174
225.52999999999997
641
123
0


In [62]:
list_scenario_7_3_e_bis = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")
    list_available_inter = get_list_og_available(date, df_inter)

    # Reminder: inter_constraints = False sets the reference to 'no constraints' and bike_proportion builds on top
    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        list_available_inter,
        inter_constraints=True,
        date=date,
        p_training_skills=0.4,
        verbose=0,
    )

    print("\n\n\n")

    list_scenario_7_3_e_bis.append(dict_results)

Date: 2024-01-15
Dropped nodes: (Total: 0)
Total available inter: 17
Total inter who worked: 16
Toal number of visited nodes: 103
Overall total commuting time: 534 minutes
Number of cars used: 13
Overall total commuting time for cars: 453 minutes (43.035 kgCO2)
Number of bikes used: 3
Overall total commuting time for bikes: 81 minutes (0.054000000000000006 kgCO2)
Total number of small breaks (<30min): 65; Total number of big breaks (>30min): 22
Average length of a day of work: 10h27




Date: 2024-01-16
Dropped nodes: (Total: 0)
Total available inter: 18
Total inter who worked: 17
Toal number of visited nodes: 111
Overall total commuting time: 658 minutes
Number of cars used: 14
Overall total commuting time for cars: 590 minutes (56.05 kgCO2)
Number of bikes used: 3
Overall total commuting time for bikes: 68 minutes (0.04533333333333334 kgCO2)
Total number of small breaks (<30min): 75; Total number of big breaks (>30min): 19
Average length of a day of work: 10h19




Date: 2024-01-17
D

### 4.3.F. Training 60%


In [191]:
list_scenario_7_3_f = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")

    # Reminder: inter_constraints = False sets the reference to 'no constraints' and bike_proportion builds on top
    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        full_inter_list,
        inter_constraints=False,
        p_training_skills=0.6,
        verbose=0,
    )

    print("\n\n\n")

    list_scenario_7_3_f.append(dict_results)

Date: 2024-01-15
Dropped nodes: (Total: 0)
Total available inter: 24
Total inter who worked: 18
Toal number of visited nodes: 103
Overall total commuting time: 549 minutes
Number of cars used: 18
Overall total commuting time for cars: 549 minutes (52.154999999999994 kgCO2)
Number of bikes used: 0
Overall total commuting time for bikes: 0 minutes (0.0 kgCO2)
Total number of small breaks (<30min): 64; Total number of big breaks (>30min): 21
Average length of a day of work: 10h00




Date: 2024-01-16
Dropped nodes: (Total: 0)
Total available inter: 24
Total inter who worked: 20
Toal number of visited nodes: 111
Overall total commuting time: 452 minutes
Number of cars used: 20
Overall total commuting time for cars: 452 minutes (42.94 kgCO2)
Number of bikes used: 0
Overall total commuting time for bikes: 0 minutes (0.0 kgCO2)
Total number of small breaks (<30min): 63; Total number of big breaks (>30min): 28
Average length of a day of work: 10h15




Date: 2024-01-17
Dropped nodes: (Total: 0

In [192]:
total_nb_com = 0
total_com_tim = 0
total_visits = 0
total_drop = 0
avg_l_day = 0
tot_breaks = 0
tot_co2 = 0
nb_cars = 0
nb_bikes = 0

list_scenario = list_scenario_7_3_f
for k in range(len(list_scenario)):
    total_com_tim += list_scenario[k]["total_commuting_time"]
    total_visits += list_scenario[k]["nb_visits"]
    total_nb_com += list_scenario[k]["nb_visits"] + list_scenario[k]["nb_inter_used"]
    total_drop += list_scenario[k]["nb_dropped_nodes"]
    avg_l_day += list_scenario[k]["avg_length_day"]
    tot_breaks += list_scenario[k]["nb_break_sup"]
    tot_co2 += list_scenario[k]["eq_co2_cars"] + list_scenario[k]["eq_co2_bikes"]
    nb_cars += list_scenario[k]["nb_cars_used"]
    nb_bikes += list_scenario[k]["nb_bikes_used"]

print(total_com_tim)
print(avg_l_day / len(list_scenario_7_1_b))
print(total_com_tim / total_nb_com)
print(tot_breaks)
print(tot_co2)
print(total_visits)
print(total_drop)
print(nb_cars)
print(nb_bikes)

2418
609.3638904954695
3.173228346456693
164
229.70999999999998
641
121
0


In [64]:
list_scenario_7_3_f_bis = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")
    list_available_inter = get_list_og_available(date, df_inter)

    # Reminder: inter_constraints = False sets the reference to 'no constraints' and bike_proportion builds on top
    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        list_available_inter,
        inter_constraints=True,
        date=date,
        p_training_skills=0.6,
        verbose=0,
    )

    print("\n\n\n")

    list_scenario_7_3_f_bis.append(dict_results)

Date: 2024-01-15
Dropped nodes: (Total: 0)
Total available inter: 17
Total inter who worked: 17
Toal number of visited nodes: 103
Overall total commuting time: 468 minutes
Number of cars used: 14
Overall total commuting time for cars: 381 minutes (36.195 kgCO2)
Number of bikes used: 3
Overall total commuting time for bikes: 87 minutes (0.057999999999999996 kgCO2)
Total number of small breaks (<30min): 65; Total number of big breaks (>30min): 21
Average length of a day of work: 10h15




Date: 2024-01-16
Dropped nodes: 17; (Total: 1)
Total available inter: 18
Total inter who worked: 18
Toal number of visited nodes: 110
Overall total commuting time: 648 minutes
Number of cars used: 14
Overall total commuting time for cars: 556 minutes (52.81999999999999 kgCO2)
Number of bikes used: 4
Overall total commuting time for bikes: 92 minutes (0.06133333333333333 kgCO2)
Total number of small breaks (<30min): 69; Total number of big breaks (>30min): 23
Average length of a day of work: 10h41




Da

## 4.4. Nurse availability (precisely its equivalent: more nurse)


In [128]:
# Reference
list_scenario_7_4_0 = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")
    list_available_inter = get_list_og_available(date, df_inter)

    # Reminder: inter_constraints = False sets the reference to 'no constraints' and bike_proportion builds on top
    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        list_available_inter,
        inter_constraints=True,
        date=date,
        verbose=0,
    )

    print("\n\n\n")

    list_scenario_7_4_0.append(dict_results)

Date: 2024-01-15
Dropped nodes: 21; 45; 62; (Total: 3)
Total available inter: 17
Total inter who worked: 16
Toal number of visited nodes: 100
Overall total commuting time: 520 minutes
Number of cars used: 13
Overall total commuting time for cars: 452 minutes (42.94 kgCO2)
Number of bikes used: 3
Overall total commuting time for bikes: 68 minutes (0.04533333333333334 kgCO2)
Total number of small breaks (<30min): 55; Total number of big breaks (>30min): 29
Average length of a day of work: 10h36




Date: 2024-01-16
Dropped nodes: 17; (Total: 1)
Total available inter: 18
Total inter who worked: 18
Toal number of visited nodes: 110
Overall total commuting time: 617 minutes
Number of cars used: 14
Overall total commuting time for cars: 532 minutes (50.54 kgCO2)
Number of bikes used: 4
Overall total commuting time for bikes: 85 minutes (0.056666666666666664 kgCO2)
Total number of small breaks (<30min): 70; Total number of big breaks (>30min): 22
Average length of a day of work: 10h25




Dat

In [129]:
total_nb_com = 0
total_com_tim = 0
total_visits = 0
total_drop = 0
avg_l_day = 0
tot_breaks = 0
tot_co2 = 0

list_scenario = list_scenario_7_4_0
for k in range(len(list_scenario)):
    total_com_tim += list_scenario[k]["total_commuting_time"]
    total_visits += list_scenario[k]["nb_visits"]
    total_nb_com += list_scenario[k]["nb_visits"] + list_scenario[k]["nb_inter_used"]
    total_drop += list_scenario[k]["nb_dropped_nodes"]
    avg_l_day += list_scenario[k]["avg_length_day"]
    tot_breaks += list_scenario[k]["nb_break_sup"]
    tot_co2 += list_scenario[k]["eq_co2_cars"] + list_scenario[k]["eq_co2_bikes"]

print(total_com_tim)
print(avg_l_day / len(list_scenario_7_1_b))
print(total_com_tim / total_nb_com)
print(tot_breaks)
print(tot_co2)
print(total_visits)

2846
459.03741496598644
4.873287671232877
113
236.41
503


### 4.4.A. One more nurse


In [130]:
list_scenario_7_4_a = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")
    list_available_inter = get_list_og_available(date, df_inter)

    # Reminder: inter_constraints = False sets the reference to 'no constraints' and bike_proportion builds on top
    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        list_available_inter,
        inter_constraints=True,
        date=date,
        nb_synth_inter=1,
        verbose=0,
    )

    print("\n\n\n")

    list_scenario_7_4_a.append(dict_results)

Date: 2024-01-15


Dropped nodes: 21; 45; 62; (Total: 3)
Total available inter: 17
Total inter who worked: 17
Toal number of visited nodes: 100
Overall total commuting time: 703 minutes
Number of cars used: 14
Overall total commuting time for cars: 525 minutes (49.875 kgCO2)
Number of bikes used: 3
Overall total commuting time for bikes: 178 minutes (0.11866666666666666 kgCO2)
Total number of small breaks (<30min): 57; Total number of big breaks (>30min): 26
Average length of a day of work: 10h09




Date: 2024-01-16
Dropped nodes: 17; (Total: 1)
Total available inter: 19
Total inter who worked: 17
Toal number of visited nodes: 110
Overall total commuting time: 649 minutes
Number of cars used: 13
Overall total commuting time for cars: 537 minutes (51.015 kgCO2)
Number of bikes used: 4
Overall total commuting time for bikes: 112 minutes (0.07466666666666667 kgCO2)
Total number of small breaks (<30min): 69; Total number of big breaks (>30min): 24
Average length of a day of work: 11h27




Date: 2024-01-17


In [131]:
total_nb_com = 0
total_com_tim = 0
total_visits = 0
total_drop = 0
avg_l_day = 0
tot_breaks = 0
tot_co2 = 0

list_scenario = list_scenario_7_4_a
for k in range(len(list_scenario)):
    total_com_tim += list_scenario[k]["total_commuting_time"]
    total_visits += list_scenario[k]["nb_visits"]
    total_nb_com += list_scenario[k]["nb_visits"] + list_scenario[k]["nb_inter_used"]
    total_drop += list_scenario[k]["nb_dropped_nodes"]
    avg_l_day += list_scenario[k]["avg_length_day"]
    tot_breaks += list_scenario[k]["nb_break_sup"]
    tot_co2 += list_scenario[k]["eq_co2_cars"] + list_scenario[k]["eq_co2_bikes"]

print(total_com_tim)
print(avg_l_day / len(list_scenario_7_1_b))
print(total_com_tim / total_nb_com)
print(tot_breaks)
print(tot_co2)
print(total_visits)

2943
451.3366596638656
5.039383561643835
103
220.81533333333334
503


### 4.4.B. Two more nurses


In [132]:
list_scenario_7_4_b = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")
    list_available_inter = get_list_og_available(date, df_inter)

    # Reminder: inter_constraints = False sets the reference to 'no constraints' and bike_proportion builds on top
    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        list_available_inter,
        inter_constraints=True,
        date=date,
        nb_synth_inter=2,
        verbose=0,
    )

    print("\n\n\n")

    list_scenario_7_4_b.append(dict_results)

Date: 2024-01-15
Dropped nodes: (Total: 0)
Total available inter: 18
Total inter who worked: 18
Toal number of visited nodes: 103
Overall total commuting time: 476 minutes
Number of cars used: 14
Overall total commuting time for cars: 370 minutes (35.15 kgCO2)
Number of bikes used: 4
Overall total commuting time for bikes: 106 minutes (0.07066666666666667 kgCO2)
Total number of small breaks (<30min): 59; Total number of big breaks (>30min): 26
Average length of a day of work: 11h20




Date: 2024-01-16
Dropped nodes: (Total: 0)
Total available inter: 20
Total inter who worked: 20
Toal number of visited nodes: 111
Overall total commuting time: 840 minutes
Number of cars used: 16
Overall total commuting time for cars: 518 minutes (49.209999999999994 kgCO2)
Number of bikes used: 4
Overall total commuting time for bikes: 322 minutes (0.21466666666666664 kgCO2)
Total number of small breaks (<30min): 70; Total number of big breaks (>30min): 21
Average length of a day of work: 9h47




Date: 

In [133]:
total_nb_com = 0
total_com_tim = 0
total_visits = 0
total_drop = 0
avg_l_day = 0
tot_breaks = 0
tot_co2 = 0

list_scenario = list_scenario_7_4_b
for k in range(len(list_scenario)):
    total_com_tim += list_scenario[k]["total_commuting_time"]
    total_visits += list_scenario[k]["nb_visits"]
    total_nb_com += list_scenario[k]["nb_visits"] + list_scenario[k]["nb_inter_used"]
    total_drop += list_scenario[k]["nb_dropped_nodes"]
    avg_l_day += list_scenario[k]["avg_length_day"]
    tot_breaks += list_scenario[k]["nb_break_sup"]
    tot_co2 += list_scenario[k]["eq_co2_cars"] + list_scenario[k]["eq_co2_bikes"]

print(total_com_tim)
print(avg_l_day / len(list_scenario_7_1_b))
print(total_com_tim / total_nb_com)
print(tot_breaks)
print(tot_co2)
print(total_visits)

3006
447.75482693077225
5.077702702702703
103
228.30966666666666
507


### 4.4.C. Three more nurses


In [134]:
list_scenario_7_4_c = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")
    list_available_inter = get_list_og_available(date, df_inter)

    # Reminder: inter_constraints = False sets the reference to 'no constraints' and bike_proportion builds on top
    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        list_available_inter,
        inter_constraints=True,
        date=date,
        nb_synth_inter=3,
        verbose=0,
    )

    print("\n\n\n")

    list_scenario_7_4_c.append(dict_results)

Date: 2024-01-15


Dropped nodes: (Total: 0)
Total available inter: 18
Total inter who worked: 18
Toal number of visited nodes: 103
Overall total commuting time: 527 minutes
Number of cars used: 15
Overall total commuting time for cars: 463 minutes (43.985 kgCO2)
Number of bikes used: 3
Overall total commuting time for bikes: 64 minutes (0.042666666666666665 kgCO2)
Total number of small breaks (<30min): 62; Total number of big breaks (>30min): 23
Average length of a day of work: 10h14




Date: 2024-01-16
Dropped nodes: (Total: 0)
Total available inter: 19
Total inter who worked: 17
Toal number of visited nodes: 111
Overall total commuting time: 685 minutes
Number of cars used: 13
Overall total commuting time for cars: 562 minutes (53.39 kgCO2)
Number of bikes used: 4
Overall total commuting time for bikes: 123 minutes (0.08199999999999999 kgCO2)
Total number of small breaks (<30min): 72; Total number of big breaks (>30min): 22
Average length of a day of work: 11h13




Date: 2024-01-17
Dropped nodes: (T

In [135]:
total_nb_com = 0
total_com_tim = 0
total_visits = 0
total_drop = 0
avg_l_day = 0
tot_breaks = 0
tot_co2 = 0

list_scenario = list_scenario_7_4_c
for k in range(len(list_scenario)):
    total_com_tim += list_scenario[k]["total_commuting_time"]
    total_visits += list_scenario[k]["nb_visits"]
    total_nb_com += list_scenario[k]["nb_visits"] + list_scenario[k]["nb_inter_used"]
    total_drop += list_scenario[k]["nb_dropped_nodes"]
    avg_l_day += list_scenario[k]["avg_length_day"]
    tot_breaks += list_scenario[k]["nb_break_sup"]
    tot_co2 += list_scenario[k]["eq_co2_cars"] + list_scenario[k]["eq_co2_bikes"]

print(total_com_tim)
print(avg_l_day / len(list_scenario_7_1_b))
print(total_com_tim / total_nb_com)
print(tot_breaks)
print(tot_co2)
print(total_visits)

2727
447.31493930905697
4.606418918918919
111
213.219
507


### 4.4.D. Four more nurses


In [138]:
list_scenario_7_4_d = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")
    list_available_inter = get_list_og_available(date, df_inter)

    # Reminder: inter_constraints = False sets the reference to 'no constraints' and bike_proportion builds on top
    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        list_available_inter,
        inter_constraints=True,
        date=date,
        nb_synth_inter=4,
        verbose=0,
    )

    print("\n\n\n")

    list_scenario_7_4_d.append(dict_results)

Date: 2024-01-15
Dropped nodes: (Total: 0)
Total available inter: 20
Total inter who worked: 17
Toal number of visited nodes: 103
Overall total commuting time: 515 minutes
Number of cars used: 14
Overall total commuting time for cars: 419 minutes (39.80499999999999 kgCO2)
Number of bikes used: 3
Overall total commuting time for bikes: 96 minutes (0.06399999999999999 kgCO2)
Total number of small breaks (<30min): 62; Total number of big breaks (>30min): 24
Average length of a day of work: 10h29




Date: 2024-01-16
Dropped nodes: (Total: 0)
Total available inter: 21
Total inter who worked: 20
Toal number of visited nodes: 111
Overall total commuting time: 636 minutes
Number of cars used: 16
Overall total commuting time for cars: 548 minutes (52.06 kgCO2)
Number of bikes used: 4
Overall total commuting time for bikes: 88 minutes (0.058666666666666666 kgCO2)
Total number of small breaks (<30min): 66; Total number of big breaks (>30min): 25
Average length of a day of work: 10h06




Date: 2

In [139]:
total_nb_com = 0
total_com_tim = 0
total_visits = 0
total_drop = 0
avg_l_day = 0
tot_breaks = 0
tot_co2 = 0

list_scenario = list_scenario_7_4_d
for k in range(len(list_scenario)):
    total_com_tim += list_scenario[k]["total_commuting_time"]
    total_visits += list_scenario[k]["nb_visits"]
    total_nb_com += list_scenario[k]["nb_visits"] + list_scenario[k]["nb_inter_used"]
    total_drop += list_scenario[k]["nb_dropped_nodes"]
    avg_l_day += list_scenario[k]["avg_length_day"]
    tot_breaks += list_scenario[k]["nb_break_sup"]
    tot_co2 += list_scenario[k]["eq_co2_cars"] + list_scenario[k]["eq_co2_bikes"]

print(total_com_tim)
print(avg_l_day / len(list_scenario_7_1_b))
print(total_com_tim / total_nb_com)
print(tot_breaks)
print(tot_co2)
print(total_visits)

2862
454.1397425636922
4.86734693877551
114
224.629
504


### 4.4.E. Five more nurses


In [140]:
list_scenario_7_4_e = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")
    list_available_inter = get_list_og_available(date, df_inter)

    # Reminder: inter_constraints = False sets the reference to 'no constraints' and bike_proportion builds on top
    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        list_available_inter,
        inter_constraints=True,
        date=date,
        nb_synth_inter=5,
        verbose=0,
    )

    print("\n\n\n")

    list_scenario_7_4_e.append(dict_results)

Date: 2024-01-15


Dropped nodes: (Total: 0)
Total available inter: 20
Total inter who worked: 17
Toal number of visited nodes: 103
Overall total commuting time: 511 minutes
Number of cars used: 15
Overall total commuting time for cars: 465 minutes (44.175 kgCO2)
Number of bikes used: 2
Overall total commuting time for bikes: 46 minutes (0.030666666666666665 kgCO2)
Total number of small breaks (<30min): 63; Total number of big breaks (>30min): 23
Average length of a day of work: 10h43




Date: 2024-01-16
Dropped nodes: (Total: 0)
Total available inter: 23
Total inter who worked: 19
Toal number of visited nodes: 111
Overall total commuting time: 582 minutes
Number of cars used: 16
Overall total commuting time for cars: 510 minutes (48.45 kgCO2)
Number of bikes used: 3
Overall total commuting time for bikes: 72 minutes (0.048 kgCO2)
Total number of small breaks (<30min): 64; Total number of big breaks (>30min): 28
Average length of a day of work: 10h52




Date: 2024-01-17
Dropped nodes: (Total: 0)
Total 

In [141]:
total_nb_com = 0
total_com_tim = 0
total_visits = 0
total_drop = 0
avg_l_day = 0
tot_breaks = 0
tot_co2 = 0

list_scenario = list_scenario_7_4_e
for k in range(len(list_scenario)):
    total_com_tim += list_scenario[k]["total_commuting_time"]
    total_visits += list_scenario[k]["nb_visits"]
    total_nb_com += list_scenario[k]["nb_visits"] + list_scenario[k]["nb_inter_used"]
    total_drop += list_scenario[k]["nb_dropped_nodes"]
    avg_l_day += list_scenario[k]["avg_length_day"]
    tot_breaks += list_scenario[k]["nb_break_sup"]
    tot_co2 += list_scenario[k]["eq_co2_cars"] + list_scenario[k]["eq_co2_bikes"]

print(total_com_tim)
print(avg_l_day / len(list_scenario_7_1_b))
print(total_com_tim / total_nb_com)
print(tot_breaks)
print(tot_co2)
print(total_visits)

2704
450.6135006634232
4.559865092748735
119
210.84533333333334
506


### 4.4.F. Ten more nurses


In [142]:
list_scenario_7_4_f = []

for date in list_dates:
    condition = c_l_need["Date"] == date
    df_need = c_l_need[condition].copy()
    df_need.drop(columns=["Date"], inplace=True)
    df_need.reset_index(drop=True, inplace=True)

    print(f"Date: {date}")
    list_available_inter = get_list_og_available(date, df_inter)

    # Reminder: inter_constraints = False sets the reference to 'no constraints' and bike_proportion builds on top
    dict_results = main_real_data_with_synth(
        df_need,
        full_time_matrix_car,
        full_time_matrix_bike,
        list_available_inter,
        inter_constraints=True,
        date=date,
        nb_synth_inter=10,
        verbose=0,
    )

    print("\n\n\n")

    list_scenario_7_4_f.append(dict_results)

Date: 2024-01-15


Dropped nodes: (Total: 0)
Total available inter: 24
Total inter who worked: 17
Toal number of visited nodes: 103
Overall total commuting time: 411 minutes
Number of cars used: 15
Overall total commuting time for cars: 374 minutes (35.53 kgCO2)
Number of bikes used: 2
Overall total commuting time for bikes: 37 minutes (0.024666666666666663 kgCO2)
Total number of small breaks (<30min): 64; Total number of big breaks (>30min): 22
Average length of a day of work: 10h35




Date: 2024-01-16
Dropped nodes: (Total: 0)
Total available inter: 25
Total inter who worked: 20
Toal number of visited nodes: 111
Overall total commuting time: 566 minutes
Number of cars used: 16
Overall total commuting time for cars: 506 minutes (48.07 kgCO2)
Number of bikes used: 4
Overall total commuting time for bikes: 60 minutes (0.039999999999999994 kgCO2)
Total number of small breaks (<30min): 68; Total number of big breaks (>30min): 23
Average length of a day of work: 9h36




Date: 2024-01-17
Dropped nodes: (Tot

In [143]:
total_nb_com = 0
total_com_tim = 0
total_visits = 0
total_drop = 0
avg_l_day = 0
tot_breaks = 0
tot_co2 = 0

list_scenario = list_scenario_7_4_f
for k in range(len(list_scenario)):
    total_com_tim += list_scenario[k]["total_commuting_time"]
    total_visits += list_scenario[k]["nb_visits"]
    total_nb_com += list_scenario[k]["nb_visits"] + list_scenario[k]["nb_inter_used"]
    total_drop += list_scenario[k]["nb_dropped_nodes"]
    avg_l_day += list_scenario[k]["avg_length_day"]
    tot_breaks += list_scenario[k]["nb_break_sup"]
    tot_co2 += list_scenario[k]["eq_co2_cars"] + list_scenario[k]["eq_co2_bikes"]

print(total_com_tim)
print(avg_l_day / len(list_scenario_7_1_b))
print(total_com_tim / total_nb_com)
print(tot_breaks)
print(tot_co2)
print(total_visits)

2323
423.7252567693744
3.8846153846153846
110
199.08266666666668
506


## 4.5. More clients, how to cope ?


In [144]:
from IPython.display import clear_output

In [145]:
def get_nb_total_drop(list_dict, exclude_weekend=False):
    n = len(list_dict)
    if exclude_weekend:
        n = len(list_dict) - 2
    overall_total_drop = 0
    for k in range(n):
        overall_total_drop += list_dict[k]["nb_dropped_nodes"]
    return overall_total_drop

In [146]:
list_dates_without_we = list_dates[
    :-2
]  # Reduce computing time and avoid weekend availability issues

### 4.5.A. Three more clients per day


In [76]:
max_nb_dropped_nodes = 2

enough_inter = False
count_inter = 0

total_drop = 0

while not enough_inter:
    clear_output(wait=True)
    print(f"Last total drop: {total_drop}")
    print(f"Current inter count: {count_inter}")
    list_scenario_7_5_a = []

    for date in list_dates_without_we:
        condition = c_l_need["Date"] == date
        df_need = c_l_need[condition].copy()
        df_need.drop(columns=["Date"], inplace=True)
        df_need.reset_index(drop=True, inplace=True)

        print(f"Date: {date}")
        list_available_inter = get_list_og_available(date, df_inter)

        # Reminder: inter_constraints = False sets the reference to 'no constraints' and bike_proportion builds on top
        dict_results = main_real_data_with_synth(
            df_need,
            full_time_matrix_car,
            full_time_matrix_bike,
            list_available_inter,
            inter_constraints=True,
            date=date,
            nb_synth_clients=3,
            nb_synth_inter=count_inter,
            verbose=0,
        )

        list_scenario_7_5_a.append(dict_results)

        print("\n\n\n")

    total_drop = get_nb_total_drop(list_scenario_7_5_a, exclude_weekend=True)
    if total_drop < max_nb_dropped_nodes:
        enough_inter = True
    else:
        count_inter += 1

print(f"Final needed extra inter: {count_inter}")

Last total drop: 0
Current inter count: 0
Date: 2024-01-15
Dropped nodes: (Total: 0)
Total available inter: 17
Total inter who worked: 16
Toal number of visited nodes: 106
Overall total commuting time: 532 minutes
Number of cars used: 13
Overall total commuting time for cars: 404 minutes (38.379999999999995 kgCO2)
Number of bikes used: 3
Overall total commuting time for bikes: 128 minutes (0.08533333333333333 kgCO2)
Total number of small breaks (<30min): 69; Total number of big breaks (>30min): 21
Average length of a day of work: 10h46




Date: 2024-01-16
Dropped nodes: 88; (Total: 1)
Total available inter: 18
Total inter who worked: 18
Toal number of visited nodes: 113
Overall total commuting time: 636 minutes
Number of cars used: 14
Overall total commuting time for cars: 511 minutes (48.545 kgCO2)
Number of bikes used: 4
Overall total commuting time for bikes: 125 minutes (0.08333333333333333 kgCO2)
Total number of small breaks (<30min): 75; Total number of big breaks (>30min): 20
A

In [193]:
total_nb_com = 0
total_com_tim = 0
total_visits = 0
total_drop = 0
avg_l_day = 0
tot_breaks = 0
tot_co2 = 0
nb_cars = 0
nb_bikes = 0

list_scenario = list_scenario_7_5_a
for k in range(len(list_scenario)):
    total_com_tim += list_scenario[k]["total_commuting_time"]
    total_visits += list_scenario[k]["nb_visits"]
    total_nb_com += list_scenario[k]["nb_visits"] + list_scenario[k]["nb_inter_used"]
    total_drop += list_scenario[k]["nb_dropped_nodes"]
    avg_l_day += list_scenario[k]["avg_length_day"]
    tot_breaks += list_scenario[k]["nb_break_sup"]
    tot_co2 += list_scenario[k]["eq_co2_cars"] + list_scenario[k]["eq_co2_bikes"]
    nb_cars += list_scenario[k]["nb_cars_used"]
    nb_bikes += list_scenario[k]["nb_bikes_used"]

print(total_com_tim)
print(avg_l_day / len(list_scenario_7_1_b))
print(total_com_tim / total_nb_com)
print(tot_breaks)
print(tot_co2)
print(total_visits)
print(total_drop)
print(nb_cars)
print(nb_bikes)

2953
436.1950396825397
4.897180762852405
94
236.48133333333334
521
1
68
14


### 4.5.B. Five more clients per day


In [77]:
max_nb_dropped_nodes = 2

enough_inter = False
count_inter = 0

total_drop = 0

while not enough_inter:
    clear_output(wait=True)
    print(f"Last total drop: {total_drop}")
    print(f"Current inter count: {count_inter}")
    list_scenario_7_5_b = []

    for date in list_dates_without_we:
        condition = c_l_need["Date"] == date
        df_need = c_l_need[condition].copy()
        df_need.drop(columns=["Date"], inplace=True)
        df_need.reset_index(drop=True, inplace=True)

        print(f"Date: {date}")
        list_available_inter = get_list_og_available(date, df_inter)

        # Reminder: inter_constraints = False sets the reference to 'no constraints' and bike_proportion builds on top
        dict_results = main_real_data_with_synth(
            df_need,
            full_time_matrix_car,
            full_time_matrix_bike,
            list_available_inter,
            inter_constraints=True,
            date=date,
            nb_synth_clients=5,
            nb_synth_inter=count_inter,
            verbose=0,
        )

        list_scenario_7_5_b.append(dict_results)

        print("\n\n\n")

    total_drop = get_nb_total_drop(list_scenario_7_5_b, exclude_weekend=True)
    if total_drop < max_nb_dropped_nodes:
        enough_inter = True
    else:
        count_inter += 1

print(f"Final needed extra inter: {count_inter}")

Last total drop: 0
Current inter count: 0
Date: 2024-01-15
Dropped nodes: (Total: 0)
Total available inter: 17
Total inter who worked: 17
Toal number of visited nodes: 108
Overall total commuting time: 556 minutes
Number of cars used: 14
Overall total commuting time for cars: 489 minutes (46.455 kgCO2)
Number of bikes used: 3
Overall total commuting time for bikes: 67 minutes (0.04466666666666667 kgCO2)
Total number of small breaks (<30min): 71; Total number of big breaks (>30min): 20
Average length of a day of work: 10h36




Date: 2024-01-16
Dropped nodes: 17; (Total: 1)
Total available inter: 18
Total inter who worked: 18
Toal number of visited nodes: 115
Overall total commuting time: 901 minutes
Number of cars used: 14
Overall total commuting time for cars: 634 minutes (60.23 kgCO2)
Number of bikes used: 4
Overall total commuting time for bikes: 267 minutes (0.178 kgCO2)
Total number of small breaks (<30min): 79; Total number of big breaks (>30min): 18
Average length of a day of wo

In [194]:
total_nb_com = 0
total_com_tim = 0
total_visits = 0
total_drop = 0
avg_l_day = 0
tot_breaks = 0
tot_co2 = 0
nb_cars = 0
nb_bikes = 0

list_scenario = list_scenario_7_5_b
for k in range(len(list_scenario)):
    total_com_tim += list_scenario[k]["total_commuting_time"]
    total_visits += list_scenario[k]["nb_visits"]
    total_nb_com += list_scenario[k]["nb_visits"] + list_scenario[k]["nb_inter_used"]
    total_drop += list_scenario[k]["nb_dropped_nodes"]
    avg_l_day += list_scenario[k]["avg_length_day"]
    tot_breaks += list_scenario[k]["nb_break_sup"]
    tot_co2 += list_scenario[k]["eq_co2_cars"] + list_scenario[k]["eq_co2_bikes"]
    nb_cars += list_scenario[k]["nb_cars_used"]
    nb_bikes += list_scenario[k]["nb_bikes_used"]

print(total_com_tim)
print(avg_l_day / len(list_scenario_7_1_b))
print(total_com_tim / total_nb_com)
print(tot_breaks)
print(tot_co2)
print(total_visits)
print(total_drop)
print(nb_cars)
print(nb_bikes)

3229
439.86069094304395
5.284779050736497
96
249.58899999999997
529
3
68
14


### 4.5.C. Ten more clients per day


In [78]:
max_nb_dropped_nodes = 2

enough_inter = False
count_inter = 3

total_drop = 0

while not enough_inter:
    clear_output(wait=True)
    print(f"Last total drop: {total_drop}")
    print(f"Current inter count: {count_inter}")
    list_scenario_7_5_c = []

    for date in list_dates_without_we:
        condition = c_l_need["Date"] == date
        df_need = c_l_need[condition].copy()
        df_need.drop(columns=["Date"], inplace=True)
        df_need.reset_index(drop=True, inplace=True)

        print(f"Date: {date}")
        list_available_inter = get_list_og_available(date, df_inter)

        # Reminder: inter_constraints = False sets the reference to 'no constraints' and bike_proportion builds on top
        dict_results = main_real_data_with_synth(
            df_need,
            full_time_matrix_car,
            full_time_matrix_bike,
            list_available_inter,
            inter_constraints=True,
            date=date,
            nb_synth_clients=10,
            nb_synth_inter=count_inter,
            verbose=0,
        )

        list_scenario_7_5_c.append(dict_results)

        print("\n\n\n")

    total_drop = get_nb_total_drop(list_scenario_7_5_c, exclude_weekend=True)
    if total_drop < max_nb_dropped_nodes:
        enough_inter = True
    else:
        count_inter += 1

print(f"Final needed extra inter: {count_inter}")

Last total drop: 2
Current inter count: 4
Date: 2024-01-15
Dropped nodes: (Total: 0)
Total available inter: 20
Total inter who worked: 19
Toal number of visited nodes: 113
Overall total commuting time: 607 minutes
Number of cars used: 17
Overall total commuting time for cars: 579 minutes (55.004999999999995 kgCO2)
Number of bikes used: 2
Overall total commuting time for bikes: 28 minutes (0.018666666666666668 kgCO2)
Total number of small breaks (<30min): 70; Total number of big breaks (>30min): 24
Average length of a day of work: 10h10




Date: 2024-01-16
Dropped nodes: 17; (Total: 1)
Total available inter: 21
Total inter who worked: 18
Toal number of visited nodes: 120
Overall total commuting time: 746 minutes
Number of cars used: 13
Overall total commuting time for cars: 596 minutes (56.61999999999999 kgCO2)
Number of bikes used: 5
Overall total commuting time for bikes: 150 minutes (0.1 kgCO2)
Total number of small breaks (<30min): 84; Total number of big breaks (>30min): 18
Averag

In [195]:
total_nb_com = 0
total_com_tim = 0
total_visits = 0
total_drop = 0
avg_l_day = 0
tot_breaks = 0
tot_co2 = 0
nb_cars = 0
nb_bikes = 0

list_scenario = list_scenario_7_5_c
for k in range(len(list_scenario)):
    total_com_tim += list_scenario[k]["total_commuting_time"]
    total_visits += list_scenario[k]["nb_visits"]
    total_nb_com += list_scenario[k]["nb_visits"] + list_scenario[k]["nb_inter_used"]
    total_drop += list_scenario[k]["nb_dropped_nodes"]
    avg_l_day += list_scenario[k]["avg_length_day"]
    tot_breaks += list_scenario[k]["nb_break_sup"]
    tot_co2 += list_scenario[k]["eq_co2_cars"] + list_scenario[k]["eq_co2_bikes"]
    nb_cars += list_scenario[k]["nb_cars_used"]
    nb_bikes += list_scenario[k]["nb_bikes_used"]

print(total_com_tim)
print(avg_l_day / len(list_scenario_7_1_b))
print(total_com_tim / total_nb_com)
print(tot_breaks)
print(tot_co2)
print(total_visits)
print(total_drop)
print(nb_cars)
print(nb_bikes)

3270
444.53206545776203
5.069767441860465
107
260.6533333333333
556
1
73
16


### 4.5.D. Twenty-five more clients per day


In [79]:
max_nb_dropped_nodes = 2

enough_inter = False
count_inter = 9

total_drop = 0

while not enough_inter:
    clear_output(wait=True)
    print(f"Last total drop: {total_drop}")
    print(f"Current inter count: {count_inter}")
    list_scenario_7_5_d = []

    for date in list_dates_without_we:
        condition = c_l_need["Date"] == date
        df_need = c_l_need[condition].copy()
        df_need.drop(columns=["Date"], inplace=True)
        df_need.reset_index(drop=True, inplace=True)

        print(f"Date: {date}")
        list_available_inter = get_list_og_available(date, df_inter)

        # Reminder: inter_constraints = False sets the reference to 'no constraints' and bike_proportion builds on top
        dict_results = main_real_data_with_synth(
            df_need,
            full_time_matrix_car,
            full_time_matrix_bike,
            list_available_inter,
            inter_constraints=True,
            date=date,
            nb_synth_clients=25,
            nb_synth_inter=count_inter,
            verbose=0,
        )

        list_scenario_7_5_d.append(dict_results)

        print("\n\n\n")

    total_drop = get_nb_total_drop(list_scenario_7_5_d, exclude_weekend=True)
    if total_drop < max_nb_dropped_nodes:
        enough_inter = True
    else:
        count_inter += 3

print(f"Final needed extra inter: {count_inter}")

Last total drop: 2
Current inter count: 12
Date: 2024-01-15
Dropped nodes: (Total: 0)
Total available inter: 24
Total inter who worked: 22
Toal number of visited nodes: 128
Overall total commuting time: 1025 minutes
Number of cars used: 17
Overall total commuting time for cars: 644 minutes (61.17999999999999 kgCO2)
Number of bikes used: 5
Overall total commuting time for bikes: 381 minutes (0.254 kgCO2)
Total number of small breaks (<30min): 82; Total number of big breaks (>30min): 24
Average length of a day of work: 10h07




Date: 2024-01-16
Dropped nodes: (Total: 0)
Total available inter: 27
Total inter who worked: 23
Toal number of visited nodes: 136
Overall total commuting time: 1032 minutes
Number of cars used: 16
Overall total commuting time for cars: 706 minutes (67.07 kgCO2)
Number of bikes used: 7
Overall total commuting time for bikes: 326 minutes (0.21733333333333332 kgCO2)
Total number of small breaks (<30min): 84; Total number of big breaks (>30min): 29
Average length of 

In [196]:
total_nb_com = 0
total_com_tim = 0
total_visits = 0
total_drop = 0
avg_l_day = 0
tot_breaks = 0
tot_co2 = 0
nb_cars = 0
nb_bikes = 0

list_scenario = list_scenario_7_5_d
for k in range(len(list_scenario)):
    total_com_tim += list_scenario[k]["total_commuting_time"]
    total_visits += list_scenario[k]["nb_visits"]
    total_nb_com += list_scenario[k]["nb_visits"] + list_scenario[k]["nb_inter_used"]
    total_drop += list_scenario[k]["nb_dropped_nodes"]
    avg_l_day += list_scenario[k]["avg_length_day"]
    tot_breaks += list_scenario[k]["nb_break_sup"]
    tot_co2 += list_scenario[k]["eq_co2_cars"] + list_scenario[k]["eq_co2_bikes"]
    nb_cars += list_scenario[k]["nb_cars_used"]
    nb_bikes += list_scenario[k]["nb_bikes_used"]

print(total_com_tim)
print(avg_l_day / len(list_scenario_7_1_b))
print(total_com_tim / total_nb_com)
print(tot_breaks)
print(tot_co2)
print(total_visits)
print(total_drop)
print(nb_cars)
print(nb_bikes)

4376
448.6480013085603
5.9537414965986395
123
311.38733333333334
631
1
83
21


### 4.5.E. Fifty more clients per day


In [80]:
max_nb_dropped_nodes = 3

enough_inter = False
count_inter = 15

total_drop = 0

while not enough_inter:
    clear_output(wait=True)
    print(f"Last total drop: {total_drop}")
    list_scenario_7_5_e = []

    for date in list_dates_without_we:
        condition = c_l_need["Date"] == date
        df_need = c_l_need[condition].copy()
        df_need.drop(columns=["Date"], inplace=True)
        df_need.reset_index(drop=True, inplace=True)

        print(f"Date: {date}")
        list_available_inter = get_list_og_available(date, df_inter)

        # Reminder: inter_constraints = False sets the reference to 'no constraints' and bike_proportion builds on top
        dict_results = main_real_data_with_synth(
            df_need,
            full_time_matrix_car,
            full_time_matrix_bike,
            list_available_inter,
            inter_constraints=True,
            date=date,
            nb_synth_clients=50,
            nb_synth_inter=count_inter,
            verbose=0,
        )

        list_scenario_7_5_e.append(dict_results)

        print("\n\n\n")

    total_drop = get_nb_total_drop(list_scenario_7_5_e, exclude_weekend=True)
    if total_drop < max_nb_dropped_nodes:
        enough_inter = True
    else:
        count_inter += 5

print(f"Final needed extra inter: {count_inter}")

Last total drop: 0
Date: 2024-01-15
Dropped nodes: (Total: 0)
Total available inter: 25
Total inter who worked: 24
Toal number of visited nodes: 153
Overall total commuting time: 1231 minutes
Number of cars used: 21
Overall total commuting time for cars: 911 minutes (86.54499999999999 kgCO2)
Number of bikes used: 3
Overall total commuting time for bikes: 320 minutes (0.21333333333333332 kgCO2)
Total number of small breaks (<30min): 105; Total number of big breaks (>30min): 24
Average length of a day of work: 10h03




Date: 2024-01-16
Dropped nodes: (Total: 0)
Total available inter: 29
Total inter who worked: 26
Toal number of visited nodes: 161
Overall total commuting time: 1341 minutes
Number of cars used: 22
Overall total commuting time for cars: 1103 minutes (104.785 kgCO2)
Number of bikes used: 4
Overall total commuting time for bikes: 238 minutes (0.15866666666666665 kgCO2)
Total number of small breaks (<30min): 106; Total number of big breaks (>30min): 29
Average length of a day

In [197]:
total_nb_com = 0
total_com_tim = 0
total_visits = 0
total_drop = 0
avg_l_day = 0
tot_breaks = 0
tot_co2 = 0
nb_cars = 0
nb_bikes = 0

list_scenario = list_scenario_7_5_e
for k in range(len(list_scenario)):
    total_com_tim += list_scenario[k]["total_commuting_time"]
    total_visits += list_scenario[k]["nb_visits"]
    total_nb_com += list_scenario[k]["nb_visits"] + list_scenario[k]["nb_inter_used"]
    total_drop += list_scenario[k]["nb_dropped_nodes"]
    avg_l_day += list_scenario[k]["avg_length_day"]
    tot_breaks += list_scenario[k]["nb_break_sup"]
    tot_co2 += list_scenario[k]["eq_co2_cars"] + list_scenario[k]["eq_co2_bikes"]
    nb_cars += list_scenario[k]["nb_cars_used"]
    nb_bikes += list_scenario[k]["nb_bikes_used"]

print(total_com_tim)
print(avg_l_day / len(list_scenario_7_1_b))
print(total_com_tim / total_nb_com)
print(tot_breaks)
print(tot_co2)
print(total_visits)
print(total_drop)
print(nb_cars)
print(nb_bikes)

5468
432.4964298986038
6.2277904328018225
134
430.7866666666667
754
3
105
19


### 4.5.F. Double clients per day


In [81]:
max_nb_dropped_nodes = 4

enough_inter = False
count_inter = 30

total_drop = 0

while not enough_inter:
    clear_output(wait=True)
    print(f"Last total drop: {total_drop}")
    list_scenario_7_5_f = []

    for date in list_dates_without_we:
        condition = c_l_need["Date"] == date
        df_need = c_l_need[condition].copy()
        df_need.drop(columns=["Date"], inplace=True)
        df_need.reset_index(drop=True, inplace=True)

        print(f"Date: {date}")
        list_available_inter = get_list_og_available(date, df_inter)

        # Reminder: inter_constraints = False sets the reference to 'no constraints' and bike_proportion builds on top
        nb_daily_synth_client = len(df_need)
        dict_results = main_real_data_with_synth(
            df_need,
            full_time_matrix_car,
            full_time_matrix_bike,
            list_available_inter,
            inter_constraints=True,
            date=date,
            nb_synth_clients=nb_daily_synth_client,
            nb_synth_inter=count_inter,
            verbose=0,
        )

        list_scenario_7_5_f.append(dict_results)

        print("\n\n\n")

    total_drop = get_nb_total_drop(list_scenario_7_5_f, exclude_weekend=True)
    if total_drop < max_nb_dropped_nodes:
        enough_inter = True
    else:
        count_inter += 5

print(f"Final needed extra inter: {count_inter}")

Last total drop: 0
Date: 2024-01-15
Dropped nodes: (Total: 0)
Total available inter: 36
Total inter who worked: 32
Toal number of visited nodes: 206
Overall total commuting time: 1878 minutes
Number of cars used: 29
Overall total commuting time for cars: 1614 minutes (153.32999999999998 kgCO2)
Number of bikes used: 3
Overall total commuting time for bikes: 264 minutes (0.176 kgCO2)
Total number of small breaks (<30min): 142; Total number of big breaks (>30min): 32
Average length of a day of work: 10h13




Date: 2024-01-16
Dropped nodes: 191; (Total: 1)
Total available inter: 39
Total inter who worked: 34
Toal number of visited nodes: 221
Overall total commuting time: 2309 minutes
Number of cars used: 26
Overall total commuting time for cars: 1786 minutes (169.67 kgCO2)
Number of bikes used: 8
Overall total commuting time for bikes: 523 minutes (0.3486666666666667 kgCO2)
Total number of small breaks (<30min): 162; Total number of big breaks (>30min): 25
Average length of a day of work:

In [198]:
total_nb_com = 0
total_com_tim = 0
total_visits = 0
total_drop = 0
avg_l_day = 0
tot_breaks = 0
tot_co2 = 0
nb_cars = 0
nb_bikes = 0

list_scenario = list_scenario_7_5_f
for k in range(len(list_scenario)):
    total_com_tim += list_scenario[k]["total_commuting_time"]
    total_visits += list_scenario[k]["nb_visits"]
    total_nb_com += list_scenario[k]["nb_visits"] + list_scenario[k]["nb_inter_used"]
    total_drop += list_scenario[k]["nb_dropped_nodes"]
    avg_l_day += list_scenario[k]["avg_length_day"]
    tot_breaks += list_scenario[k]["nb_break_sup"]
    tot_co2 += list_scenario[k]["eq_co2_cars"] + list_scenario[k]["eq_co2_bikes"]
    nb_cars += list_scenario[k]["nb_cars_used"]
    nb_bikes += list_scenario[k]["nb_bikes_used"]

print(total_com_tim)
print(avg_l_day / len(list_scenario_7_1_b))
print(total_com_tim / total_nb_com)
print(tot_breaks)
print(tot_co2)
print(total_visits)
print(total_drop)
print(nb_cars)
print(nb_bikes)

10122
421.60229237463005
8.65128205128205
142
773.6779999999999
1012
2
132
26
